In [118]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Eric Meinhardt / emeinhardt@ucsd.edu**

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span><ul class="toc-item"><li><span><a href="#Preprocessing-steps" data-toc-modified-id="Preprocessing-steps-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Preprocessing steps</a></span></li><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Dependencies</a></span></li><li><span><a href="#Outputs" data-toc-modified-id="Outputs-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Outputs</a></span></li></ul></li><li><span><a href="#Imports-/-loading-data" data-toc-modified-id="Imports-/-loading-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports / loading data</a></span><ul class="toc-item"><li><span><a href="#Overview-of-what's-in-the-2003-release-of-Switchboard" data-toc-modified-id="Overview-of-what's-in-the-2003-release-of-Switchboard-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Overview of what's in the 2003 release of Switchboard</a></span></li><li><span><a href="#Retrieving-files-from-conversation-ids" data-toc-modified-id="Retrieving-files-from-conversation-ids-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Retrieving files from conversation ids</a></span></li></ul></li><li><span><a href="#Parsing-and-pre-processing" data-toc-modified-id="Parsing-and-pre-processing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Parsing and pre-processing</a></span><ul class="toc-item"><li><span><a href="#Parsing-the-utterance-files" data-toc-modified-id="Parsing-the-utterance-files-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Parsing the utterance files</a></span></li><li><span><a href="#Parsing-the-word-files" data-toc-modified-id="Parsing-the-word-files-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Parsing the word files</a></span></li><li><span><a href="#Organizing-all-utterances-and-some-summary-stats" data-toc-modified-id="Organizing-all-utterances-and-some-summary-stats-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Organizing all utterances and some summary stats</a></span><ul class="toc-item"><li><span><a href="#Summary" data-toc-modified-id="Summary-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Summary</a></span></li></ul></li></ul></li><li><span><a href="#Comparison-with-Fisher" data-toc-modified-id="Comparison-with-Fisher-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Comparison with Fisher</a></span></li><li><span><a href="#Relating-the-word-and-utterance-relations" data-toc-modified-id="Relating-the-word-and-utterance-relations-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Relating the word and utterance relations</a></span><ul class="toc-item"><li><span><a href="#Mapping-each-utterance-ID-to-a-sequence-of-word-relations" data-toc-modified-id="Mapping-each-utterance-ID-to-a-sequence-of-word-relations-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Mapping each utterance ID to a sequence of word relations</a></span></li><li><span><a href="#Mapping-each-utterance-ID-to-an-orthographic-word-sequence-+-word-duration-sequence-pair" data-toc-modified-id="Mapping-each-utterance-ID-to-an-orthographic-word-sequence-+-word-duration-sequence-pair-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Mapping each utterance ID to an orthographic word sequence + word duration sequence pair</a></span></li><li><span><a href="#Making-a-word-relation-for-confusability-analysis" data-toc-modified-id="Making-a-word-relation-for-confusability-analysis-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Making a word relation for confusability analysis</a></span></li><li><span><a href="#Identifying-n-gram-contexts" data-toc-modified-id="Identifying-n-gram-contexts-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Identifying n-gram contexts</a></span></li></ul></li><li><span><a href="#Speech-rates-by-speaker" data-toc-modified-id="Speech-rates-by-speaker-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Speech rates by speaker</a></span></li><li><span><a href="#Word-Durations" data-toc-modified-id="Word-Durations-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Word Durations</a></span></li><li><span><a href="#Applying-Exclusion-Criteria" data-toc-modified-id="Applying-Exclusion-Criteria-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Applying Exclusion Criteria</a></span></li><li><span><a href="#Writing-things-to-file" data-toc-modified-id="Writing-things-to-file-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Writing things to file</a></span><ul class="toc-item"><li><span><a href="#Utterance-relation" data-toc-modified-id="Utterance-relation-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Utterance relation</a></span></li><li><span><a href="#Word-relation" data-toc-modified-id="Word-relation-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>Word relation</a></span></li><li><span><a href="#Corpus-for-a-language-model" data-toc-modified-id="Corpus-for-a-language-model-9.3"><span class="toc-item-num">9.3&nbsp;&nbsp;</span>Corpus for a language model</a></span></li><li><span><a href="#Vocabulary-for-a-language-model" data-toc-modified-id="Vocabulary-for-a-language-model-9.4"><span class="toc-item-num">9.4&nbsp;&nbsp;</span>Vocabulary for a language model</a></span></li><li><span><a href="#Word-analysis-relation" data-toc-modified-id="Word-analysis-relation-9.5"><span class="toc-item-num">9.5&nbsp;&nbsp;</span>Word analysis relation</a></span></li><li><span><a href="#N-gram-contexts" data-toc-modified-id="N-gram-contexts-9.6"><span class="toc-item-num">9.6&nbsp;&nbsp;</span>N-gram contexts</a></span></li></ul></li></ul></div>

# Overview

The main goal of this notebook is to produce a version of the 2003 release of the Switchboard corpus whose vocabulary has been normalized with respect to the Fisher corpus. The motivation for doing this is applying a language model trained on (a slightly processed version of) the Fisher corpus to Switchboard.

The second goal of this notebook is to annotate word tokens and utterance tokens with exclusion criteria for a model of reduction.

## Preprocessing steps

In the service of the main goal, 
 1. Interrupted or broken-off wordforms - wordforms that speakers didn't finish production of or that they resumed production somewhere in the middle of have been replaced with `<rem>`.
 2. Non-speech noises (e.g. `[laughter]` or `[silence]`) are removed. Note that 'utterances' that in the original contained only things like this are not included in any post-processed data meant to be input to a language model.
 3. Words that transcribers weren't sure of(?) or that seemed like (whole words but) production errors or new coinages are apparently transcribed in curly braces. All such curly braces have been removed.
 4. The following tokens appear in the corpus with an underscore:
  - `{'<b_aside>', '<e_aside>', 'about_1', 'because_1', 'depends_1', 'especially_1', 'okay_1', "them's_1", 'them_1', "them_1's"}`
  - `_aside` tokens have been removed altogether and the trailing `_1` has been removed from all others.
 5. All characters are lower-cased.

In [2]:
import re

In [117]:
from collections import OrderedDict

In [3]:
from more_itertools import replace
from funcy import compose

In [4]:
# unk = '<unk>'
unk = '<rem>'

In [5]:
interrupted_word_pattern = ".*-$"
resumed_word_pattern = "^-.*"

def isInterrupted(wordform):
    return re.match(interrupted_word_pattern, wordform) is not None

def isResumed(wordform):
    return re.match(resumed_word_pattern, wordform) is not None

def isBroken(wordform):
    return isInterrupted(wordform) or isResumed(wordform)

def hasBrokenWords(speech):
    speech_word_seq = speech.split(' ')
    broken_words = list(filter(isBroken, speech_word_seq))
    return len(broken_words) > 0

def remove_broken_words(speech, insertUnk=True):
    if insertUnk:
        replacement = unk
    else:
        replacement = ""
        
    speech_word_seq = speech.split(' ')
    speech_word_seq_fixed = ' '.join(replace(speech_word_seq, isBroken, (replacement,)))
    speech_fixed = ' '.join([w for w in speech_word_seq_fixed.split(' ') if len(w) > 0])
    return speech_fixed

remove_broken_words("we[ll]- well in our area we just introduced the um citywide")
remove_broken_words('[noise] -[o]kay')
remove_broken_words("o[kay]- [noise] -[o]kay well i think this is going to probably be the most difficult topic that i've discussed [laughter-so] [laughter-far] yes well have you returned anything lately")
remove_broken_words("well i just buy those California peeled -[to]ma[toes]- tomatoes")

'<rem> well in our area we just introduced the um citywide'

'[noise] <rem>'

"<rem> [noise] <rem> well i think this is going to probably be the most difficult topic that i've discussed [laughter-so] [laughter-far] yes well have you returned anything lately"

'well i just buy those California peeled <rem> tomatoes'

In [6]:
def remove_non_speech(speech):
    speech_filtered_words = [w for w in speech.split(' ') if len(w) > 0 and w[0] != '[' and w[-1] != ']']
    speech_out = ' '.join(speech_filtered_words)
    speech_out = speech_out.rstrip()
    return speech_out

remove_non_speech('[noise] -[o]kay')
remove_non_speech("o[kay]- [noise] -[o]kay well i think this is going to probably be the most difficult topic that i've discussed [laughter-so] [laughter-far] yes well have you returned anything lately")
remove_non_speech("n[ot]- not big[oted]- or very m[ore]- m[ore]- you know more in the twentieth century [laughter-now] let's say or or very [vocalized-noise] {unbigoted} and they even have added uh this uh other")

'-[o]kay'

"o[kay]- -[o]kay well i think this is going to probably be the most difficult topic that i've discussed yes well have you returned anything lately"

"n[ot]- not big[oted]- or very m[ore]- m[ore]- you know more in the twentieth century let's say or or very {unbigoted} and they even have added uh this uh other"

In [7]:
def hasCurlyBraces(wordform):
    return '{' in wordform or '}' in wordform

def isCurlyBraced(wordform):
    if len(wordform) == 0:
        return False
    return wordform[0] == '{' and wordform[-1] == '}'

def removeCurlyBraces(wordform):
    if not isCurlyBraced(wordform):
        return wordform
    return wordform[1:-1]

def remove_curly_braces(speech):
    speech_word_seq = speech.split(' ')
    speech_word_seq_fixed = ' '.join(list(map(removeCurlyBraces, speech_word_seq)))
    speech_fixed = ' '.join([w for w in speech_word_seq_fixed.split(' ') if len(w) > 0])
    return speech_fixed

remove_curly_braces("n[ot]- not big[oted]- or very m[ore]- m[ore]- you know more in the twentieth century [laughter-now] let's say or or very [vocalized-noise] {unbigoted} and they even have added uh this uh other")
remove_curly_braces("yeah yes i i know there's a long scientific name but it's like {polytechnochloride} and all that fun stuff um i like")
remove_curly_braces("well it it it'll it'll catch on pretty soon i'll i'll i'll {betcha} i i have a sixteen year old almost sixteen year old Golden Retriever and she's gone the other way because she's so old she doesn't have much control")
remove_curly_braces("that works all right th[ey]- they don't uh w[ant]- want the mothers to be with the little {fishies}")

"n[ot]- not big[oted]- or very m[ore]- m[ore]- you know more in the twentieth century [laughter-now] let's say or or very [vocalized-noise] unbigoted and they even have added uh this uh other"

"yeah yes i i know there's a long scientific name but it's like polytechnochloride and all that fun stuff um i like"

"well it it it'll it'll catch on pretty soon i'll i'll i'll betcha i i have a sixteen year old almost sixteen year old Golden Retriever and she's gone the other way because she's so old she doesn't have much control"

"that works all right th[ey]- they don't uh w[ant]- want the mothers to be with the little fishies"

In [8]:
def hasUnderscore(wordform):
    return '_' in wordform

def fixUnderscore(wordform):
    if not hasUnderscore(wordform):
        return wordform
    fixed = wordform.replace('_1', '')
    return fixed

def fix_underscores(speech):
    speech_word_seq = speech.split(' ')
    speech_word_seq_filtered = [w for w in speech_word_seq if w != '<b_aside>' and w != '<e_aside>']
    speech_word_seq_fixed = ' '.join(list(map(fixUnderscore, speech_word_seq_filtered)))
    speech_fixed = ' '.join([w for w in speech_word_seq_fixed.split(' ') if len(w) > 0])
    return speech_fixed

fix_underscores("because_1 yes dogs need training but cats usually don't even need any training seem like they automatically go to their little litter box they think yeah")
fix_underscores('to feed me or to pick me up and love me <b_aside> yes you sweet thing <e_aside>')

"because yes dogs need training but cats usually don't even need any training seem like they automatically go to their little litter box they think yeah"

'to feed me or to pick me up and love me yes you sweet thing'

In [9]:
def lowercase(speech):
    return speech.lower()

In [10]:
process = compose(lowercase, fix_underscores, remove_curly_braces, remove_non_speech, remove_broken_words)

process("we[ll]- well in our area we just introduced the um citywide")
process('[noise] -[o]kay')
process("o[kay]- [noise] -[o]kay well i think this is going to probably be the most difficult topic that i've discussed [laughter-so] [laughter-far] yes well have you returned anything lately")
process("well i just buy those California peeled -[to]ma[toes]- tomatoes")
process("n[ot]- not big[oted]- or very m[ore]- m[ore]- you know more in the twentieth century [laughter-now] let's say or or very [vocalized-noise] {unbigoted} and they even have added uh this uh other")
process("yeah yes i i know there's a long scientific name but it's like {polytechnochloride} and all that fun stuff um i like")
process("well it it it'll it'll catch on pretty soon i'll i'll i'll {betcha} i i have a sixteen year old almost sixteen year old Golden Retriever and she's gone the other way because she's so old she doesn't have much control")
process("that works all right th[ey]- they don't uh w[ant]- want the mothers to be with the little {fishies}")
process("because_1 yes dogs need training but cats usually don't even need any training seem like they automatically go to their little litter box they think yeah")
process('to feed me or to pick me up and love me <b_aside> yes you sweet thing <e_aside>')

'<rem> well in our area we just introduced the um citywide'

'<rem>'

"<rem> <rem> well i think this is going to probably be the most difficult topic that i've discussed yes well have you returned anything lately"

'well i just buy those california peeled <rem> tomatoes'

"<rem> not <rem> or very <rem> <rem> you know more in the twentieth century let's say or or very unbigoted and they even have added uh this uh other"

"yeah yes i i know there's a long scientific name but it's like polytechnochloride and all that fun stuff um i like"

"well it it it'll it'll catch on pretty soon i'll i'll i'll betcha i i have a sixteen year old almost sixteen year old golden retriever and she's gone the other way because she's so old she doesn't have much control"

"that works all right <rem> they don't uh <rem> want the mothers to be with the little fishies"

"because yes dogs need training but cats usually don't even need any training seem like they automatically go to their little litter box they think yeah"

'to feed me or to pick me up and love me yes you sweet thing'

## Dependencies
 - `more_itertools`, `funcy`, `joblib` and Unix shell command cell/line magics are used throughout, though none have essential functionality that couldn't relatively easily be replaced by something else if necessary.

## Outputs

If run successfully, this notebook will create nine files as outputs:
 1. A .json file containing a list of objects (Python dictionaries), where each object is a finitary relation describing an utterance (and associated metadata) in the Switchboard corpus.
 2. A .json file containing a list of objects (Python dictionaries), where each object is a finitary relation describing a wordform token (and associated metadata) in the Switchboard corpus.
 3. A .txt file containing one utterance from Switchboard per line.
 4. A .txt file containing the vocabulary (one wordform per line) of the previous file.
 5. A .json file containing a further-annotated version of the word relation (#2 above) for other research purposes.
 6. A set of .txt files containing the unigram, bigram, trigram, and fourgram contexts that occur in file #3.

# Imports / loading data

In [11]:
import os
import csv
import json

In [130]:
from tqdm import tqdm

In [12]:
from itertools import filterfalse, chain

In [13]:
from collections import Counter

In [14]:
os.getcwd()

'/mnt/cube/home/AD/emeinhar/switchboard-lm'

In [15]:
switchboard_lm_dir = '/mnt/cube/home/AD/emeinhar/switchboard-lm'

In [16]:
from joblib import Parallel, delayed

J = -1
BACKEND = 'multiprocessing'
# BACKEND = 'loky'
V = 10
PREFER = 'processes'
# PREFER = 'threads'

def par(gen_expr):
    return Parallel(n_jobs=J, backend=BACKEND, verbose=V, prefer=PREFER)(gen_expr)

## Overview of what's in the 2003 release of Switchboard

See https://www.isip.piconepress.com/projects/switchboard/.

In [17]:
swbd2003_dir = "/mnt/truffle/corpora/switchboard_word_alignments/swb_ms98_transcriptions"

In [18]:
os.chdir(swbd2003_dir)
os.listdir()

['20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 'AAREADME.text',
 'sw-ms98-dict.text']

In [19]:
%cat -n AAREADME.text | head -100

     1	This release contains manually corrected word alignments and transcriptions,
     2	and an updated dictionary.  
     3	
     4	Word alignments were generated by using the most recent release of the
     5	transcriptions and performing forced alignments using ISIP's Hub 5E
     6	recognition system (which is also publicly available). These
     7	alignments were manually reviewed for accurate discrimination
     8	between speech and non-speech.
     9	
    10	The released data is available at:
    11	
    12	 http://www.isip.msstate.edu/projects/switchboard/releases/switchboard_word_alignments.tar.gz
    13	
    14	As always, let us know if you have any questions. 
    15	
    16	Files contained in this release:
    17	
    18	2001
    19	 sw2001A-ms98-a-trans.text
    20	 sw2001A-ms98-a-word.text
    21	 sw2001B-ms98-a-trans.text
    22	 sw2001B-ms98-a-word.text
    23	2005
    24	 sw2005A-ms98-a-trans.text
    25	 sw2005A-ms98-a-word.text
    26	 sw2005B-ms98-a-trans.text
    

In [20]:
swbd2003_folders = list(filterfalse(lambda s: s[-5:] == '.text', os.listdir()))
swbd2003_folders

['20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49']

In [21]:
%ls 20/2001

sw2001A-ms98-a-trans.text*  sw2001B-ms98-a-trans.text*
sw2001A-ms98-a-word.text*   sw2001B-ms98-a-word.text*


In [22]:
# swbd2003_files = []
conversation_ids = []
for folder in swbd2003_folders:
    os.chdir(folder)
    conversation_ids.extend(os.listdir())
    os.chdir('..')
len(conversation_ids)
conversation_ids[:30]

2438

['2001',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2012',
 '2013',
 '2014',
 '2015',
 '2017',
 '2018',
 '2019',
 '2020',
 '2022',
 '2023',
 '2024',
 '2025',
 '2027',
 '2028',
 '2032',
 '2035',
 '2036',
 '2038',
 '2039',
 '2040',
 '2041',
 '2044',
 '2045']

In [23]:
%cat -n ./20/2001/sw2001A-ms98-a-trans.text | head -50

     1	sw2001A-ms98-a-0001 0.000000 0.977625 [silence]
     2	sw2001A-ms98-a-0002 0.977625 11.561375 hi um yeah i'd like to talk about how you dress for work and and um what do you normally what type of outfit do you normally have to wear
     3	sw2001A-ms98-a-0003 11.561375 19.804875 [silence]
     4	sw2001A-ms98-a-0004 19.804875 21.312375 um-hum
     5	sw2001A-ms98-a-0005 21.312375 27.362000 [silence]
     6	sw2001A-ms98-a-0006 27.362000 28.932750 and is
     7	sw2001A-ms98-a-0007 28.932750 33.898250 [silence]
     8	sw2001A-ms98-a-0008 33.898250 40.123250 right right is there is there um an[y]- is there a like a code of dress where you work do they ask
     9	sw2001A-ms98-a-0009 40.123250 41.552625 right
    10	sw2001A-ms98-a-0010 41.552625 50.129500 [silence]
    11	sw2001A-ms98-a-0011 50.129500 51.567500 [noise] right
    12	sw2001A-ms98-a-0012 51.567500 56.957000 [silence]
    13	sw2001A-ms98-a-0013 56.957000 65.593625 right right and does it does it change i guess um you can can

In [24]:
%cat -n ./20/2001/sw2001A-ms98-a-word.text | head -100

     1	sw2001A-ms98-a-0001 0.000000 0.977625 [silence]
     2	sw2001A-ms98-a-0002 0.977625 1.215250 [silence]
     3	sw2001A-ms98-a-0002 1.215250 1.724625 hi
     4	sw2001A-ms98-a-0002 1.724625 2.273625 [silence]
     5	sw2001A-ms98-a-0002 2.273625 2.927625 um
     6	sw2001A-ms98-a-0002 2.927625 3.221500 [silence]
     7	sw2001A-ms98-a-0002 3.221500 3.661750 yeah
     8	sw2001A-ms98-a-0002 3.661750 3.957625 i'd
     9	sw2001A-ms98-a-0002 3.957625 4.107625 like
    10	sw2001A-ms98-a-0002 4.107625 4.267625 to
    11	sw2001A-ms98-a-0002 4.267625 4.527625 talk
    12	sw2001A-ms98-a-0002 4.527625 4.941625 about
    13	sw2001A-ms98-a-0002 4.941625 5.126125 [silence]
    14	sw2001A-ms98-a-0002 5.126125 5.307625 how
    15	sw2001A-ms98-a-0002 5.307625 5.437625 you
    16	sw2001A-ms98-a-0002 5.437625 5.735375 dress
    17	sw2001A-ms98-a-0002 5.735375 5.901125 [silence]
    18	sw2001A-ms98-a-0002 5.901125 6.077625 for
    19	sw2001A-ms98-a-0002 6.077625 6.477625 work
    20	sw2001A-ms98-a-0002 6

For each of ≈2400 conversations, there is 
 - a sequence of utterances for speaker A.
 - a sequence of utterances for speaker B.
 - a sequence of orthographic wordforms (and non-speech events) for speaker A.
 - a sequence of orthographic wordforms (and non-speech events) for speaker B.
 
For each utterance, there is a start and end time.

For each orthographic wordform or non-speech event, there is a duration.

## Retrieving files from conversation ids

In [25]:
%ls 20/2001

sw2001A-ms98-a-trans.text*  sw2001B-ms98-a-trans.text*
sw2001A-ms98-a-word.text*   sw2001B-ms98-a-word.text*


In [26]:
conversation_ids = set(conversation_ids)

In [27]:
def getDir(conversation_id):
    assert conversation_id in conversation_ids, '{0} is not a conversation id'.format(conversation_id)
    firstTwoDigits = conversation_id[:2]
    return os.path.join(swbd2003_dir, firstTwoDigits + '/' + conversation_id)

def getFilenames(conversation_id):
    assert conversation_id in conversation_ids, '{0} is not a conversation id'.format(conversation_id)
    return os.listdir(getDir(conversation_id))

getDir('2045')
getFilenames('2045')

'/mnt/truffle/corpora/switchboard_word_alignments/swb_ms98_transcriptions/20/2045'

['sw2045A-ms98-a-trans.text',
 'sw2045A-ms98-a-word.text',
 'sw2045B-ms98-a-trans.text',
 'sw2045B-ms98-a-word.text']

In [28]:
def getUtteranceFP(conversation_id, speaker):
    fp_prefix = getDir(conversation_id)
    
    assert speaker in {'A', 'B'}, 'Speaker must be either "A" or "B", got {0}'.format(speaker)
    
    fn_prefix = 'sw'
    fn_suffix = '-ms98-a-trans.text'
    fn = fn_prefix + conversation_id + speaker + fn_suffix
    assert fn in set(getFilenames(conversation_id))
    
    return os.path.join(fp_prefix, fn)

def getWordFP(conversation_id, speaker):
    fp_prefix = getDir(conversation_id)
    
    assert speaker in {'A', 'B'}, 'Speaker must be either "A" or "B", got {0}'.format(speaker)
    
    fn_prefix = 'sw'
    fn_suffix = '-ms98-a-word.text'
    fn = fn_prefix + conversation_id + speaker + fn_suffix
    assert fn in set(getFilenames(conversation_id))
    
    return os.path.join(fp_prefix, fn)

getUtteranceFP('2045','A')
getWordFP('2045','A')

'/mnt/truffle/corpora/switchboard_word_alignments/swb_ms98_transcriptions/20/2045/sw2045A-ms98-a-trans.text'

'/mnt/truffle/corpora/switchboard_word_alignments/swb_ms98_transcriptions/20/2045/sw2045A-ms98-a-word.text'

In [29]:
%cat -n /mnt/truffle/corpora/switchboard_word_alignments/swb_ms98_transcriptions/20/2045/sw2045A-ms98-a-trans.text | head -10

     1	sw2045A-ms98-a-0001 0.000000 1.411625 all right
     2	sw2045A-ms98-a-0002 1.411625 3.668625 i play volleyball and
     3	sw2045A-ms98-a-0003 3.668625 6.593875 softball and ceramics
     4	sw2045A-ms98-a-0004 6.593875 19.817125 [silence]
     5	sw2045A-ms98-a-0005 19.817125 21.945500 i didn't think of working out as a hobby
     6	sw2045A-ms98-a-0006 21.945500 22.259750 [laughter]
     7	sw2045A-ms98-a-0007 22.259750 25.479625 or playing with the computer i do both at home
     8	sw2045A-ms98-a-0008 25.479625 35.911625 [silence]
     9	sw2045A-ms98-a-0009 35.911625 43.178125 uh we've gotten a little Atari computer uh husband describes it as a a computer with training wheels
    10	sw2045A-ms98-a-0010 43.178125 44.533500 uh


# Parsing and pre-processing

## Parsing the utterance files

In [30]:
def parse_swbd_utterance_line(line):
    parts = line.split(' ')
#     parts = [e for e in line.split(' ') if e != '']
#     return parts
    utterance_id = parts[0]
    
    start = float(parts[1])
    end = float(parts[2])
    duration = end - start
    
    if len(parts) > 3:
        speech_raw_words = parts[3:]
        speech_raw = ' '.join( speech_raw_words )
    else:
#         speech_raw_words = 
        speech_raw = ''
    
    if speech_raw == '':
        speech_OnlyWords = ''
        has_word_internal_brackets = False
    else:
        speech_wordforms = speech_raw.split(' ')
        speech_wordforms = [w for w in speech_wordforms if len(w) > 0 and w[0] != '[' and w[-1] != ']']
        speech_OnlyWords = ' '.join(speech_wordforms)
        hasBrackets = lambda w: "[" in w or "]" in w
        has_word_internal_brackets = True if any(list(map(hasBrackets, speech_wordforms))) else False
        speech = process(speech_raw)
#         if any(list(map(hasBrackets, speech_wordforms))):
#             raise Exception("Word-internal brackets found in\n\t{0}".format(speech_OnlyWords))
    
#     has_words = False if len(speech_raw) == 0 or (speech_raw[0] == '[' and speech_raw[-1] == ']') else True
    has_words = False if speech == '' else True
    
    line_rel = {'utterance_id':utterance_id,
                'conversation_id':utterance_id[2:6],
                'speaker':utterance_id[6:7],
                'start':start,
                'end':end,
                'duration':duration,
                'speech_raw':speech_raw,
#                 'speech_words':speech_OnlyWords,
                'speech':speech,
#                 'n_words':len(speech_OnlyWords.split(' ')),
                'n_words':0 if not has_words else len(speech.split(' ')),
                'has_words':has_words,
                'has_word_internal_brackets':has_word_internal_brackets
#                 'speech_noquotes':speech_noquotes
               }
    
    return line_rel

In [31]:
def read_swbd_file(swbd_fp):
    lines = []
    with open(swbd_fp, 'r') as the_file:
        for line in the_file:
            lines.append(line.rstrip())
    return lines

def get_utterance_relations(conversation_id, speaker):
    utterance_fp = getUtteranceFP(conversation_id, speaker)
    file_lines = read_swbd_file(utterance_fp)
    parsed_rel = tuple([parse_swbd_utterance_line(l) for l in file_lines])
    return parsed_rel

In [32]:
utts_2045A = get_utterance_relations('2045', 'A')
utts_2045A[:10]

({'utterance_id': 'sw2045A-ms98-a-0001',
  'conversation_id': '2045',
  'speaker': 'A',
  'start': 0.0,
  'end': 1.411625,
  'duration': 1.411625,
  'speech_raw': 'all right',
  'speech': 'all right',
  'n_words': 2,
  'has_words': True,
  'has_word_internal_brackets': False},
 {'utterance_id': 'sw2045A-ms98-a-0002',
  'conversation_id': '2045',
  'speaker': 'A',
  'start': 1.411625,
  'end': 3.668625,
  'duration': 2.257,
  'speech_raw': 'i play volleyball and',
  'speech': 'i play volleyball and',
  'n_words': 4,
  'has_words': True,
  'has_word_internal_brackets': False},
 {'utterance_id': 'sw2045A-ms98-a-0003',
  'conversation_id': '2045',
  'speaker': 'A',
  'start': 3.668625,
  'end': 6.593875,
  'duration': 2.9252499999999997,
  'speech_raw': 'softball and ceramics',
  'speech': 'softball and ceramics',
  'n_words': 3,
  'has_words': True,
  'has_word_internal_brackets': False},
 {'utterance_id': 'sw2045A-ms98-a-0004',
  'conversation_id': '2045',
  'speaker': 'A',
  'start': 6.

## Parsing the word files

In [33]:
%cat -n ./20/2001/sw2001A-ms98-a-word.text | head -10

     1	sw2001A-ms98-a-0001 0.000000 0.977625 [silence]
     2	sw2001A-ms98-a-0002 0.977625 1.215250 [silence]
     3	sw2001A-ms98-a-0002 1.215250 1.724625 hi
     4	sw2001A-ms98-a-0002 1.724625 2.273625 [silence]
     5	sw2001A-ms98-a-0002 2.273625 2.927625 um
     6	sw2001A-ms98-a-0002 2.927625 3.221500 [silence]
     7	sw2001A-ms98-a-0002 3.221500 3.661750 yeah
     8	sw2001A-ms98-a-0002 3.661750 3.957625 i'd
     9	sw2001A-ms98-a-0002 3.957625 4.107625 like
    10	sw2001A-ms98-a-0002 4.107625 4.267625 to


In [34]:
def parse_swbd_word_line(line):
#     parts = line.split(' ')
    #SOME of the word files are tab separated
    #SOME use four spaces
    #SOME use one space
    # !!!!! who the hell let this happen ???
    if '\t' in line:
        parts = line.split('\t')
    else:
        parts = [e for e in line.split(' ') if e != '']
#     return parts
    utterance_id = parts[0]
    
    start = float(parts[1])
    end = float(parts[2])
    duration = end - start
    
    if len(parts) > 3:
        speech_raw_words = parts[3:]
        speech_raw = ' '.join( speech_raw_words )
    else:
#         speech_raw_words = 
        speech_raw = ''
    
    if speech_raw == '':
        speech_OnlyWords = ''
        has_word_internal_brackets = False
    else:
        speech_wordforms = speech_raw.split(' ')
        speech_wordforms = [w for w in speech_wordforms if len(w) > 0 and w[0] != '[' and w[-1] != ']']
        speech_OnlyWords = ' '.join(speech_wordforms)
        hasBrackets = lambda w: "[" in w or "]" in w
        has_word_internal_brackets = True if any(list(map(hasBrackets, speech_wordforms))) else False
        speech = process(speech_raw)
#         if any(list(map(hasBrackets, speech_wordforms))):
#             raise Exception("Word-internal brackets found in\n\t{0}".format(speech_OnlyWords))
    
#     has_words = False if len(speech_raw) == 0 or (speech_raw[0] == '[' and speech_raw[-1] == ']') else True
    has_words = False if speech == '' else True
    
    line_rel = {'utterance_id':utterance_id,
                'conversation_id':utterance_id[2:6],
                'speaker':utterance_id[6:7],
                'start':start,
                'end':end,
                'duration':duration,
                'speech_raw':speech_raw,
#                 'speech_words':speech_OnlyWords,
                'speech':speech,
                'has_words':has_words,
                'has_word_internal_brackets':has_word_internal_brackets
#                 'speech_noquotes':speech_noquotes
               }
    
    return line_rel

In [35]:
def get_word_relations(conversation_id, speaker):
    word_fp = getWordFP(conversation_id, speaker)
    file_lines = read_swbd_file(word_fp)
    parsed_rel = tuple([parse_swbd_word_line(l) for l in file_lines])
    return parsed_rel

In [36]:
words_2045A = get_word_relations('2045', 'A')
words_2045A[:10]

({'utterance_id': 'sw2045A-ms98-a-0001',
  'conversation_id': '2045',
  'speaker': 'A',
  'start': 0.0,
  'end': 0.47125,
  'duration': 0.47125,
  'speech_raw': '[silence]',
  'speech': '',
  'has_words': False,
  'has_word_internal_brackets': False},
 {'utterance_id': 'sw2045A-ms98-a-0001',
  'conversation_id': '2045',
  'speaker': 'A',
  'start': 0.47125,
  'end': 0.640625,
  'duration': 0.169375,
  'speech_raw': 'all',
  'speech': 'all',
  'has_words': True,
  'has_word_internal_brackets': False},
 {'utterance_id': 'sw2045A-ms98-a-0001',
  'conversation_id': '2045',
  'speaker': 'A',
  'start': 0.640625,
  'end': 0.909125,
  'duration': 0.26849999999999996,
  'speech_raw': 'right',
  'speech': 'right',
  'has_words': True,
  'has_word_internal_brackets': False},
 {'utterance_id': 'sw2045A-ms98-a-0001',
  'conversation_id': '2045',
  'speaker': 'A',
  'start': 0.909125,
  'end': 1.411625,
  'duration': 0.5025,
  'speech_raw': '[silence]',
  'speech': '',
  'has_words': False,
  'has_

In [37]:
# words = [get_word_relations(convo_id, spkr) for convo_id in conversation_ids for spkr in ('A','B')]

#~6s w/ J = 30 on wittgenstein
words = par(delayed(get_word_relations)(convo_id, spkr) for convo_id in conversation_ids for spkr in ('A','B'))
len(words)

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0479s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  49 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 200 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 336 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 488 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 808 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 1160 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 1344 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]

4876

In [38]:
words = list(chain.from_iterable(words))
len(words)

4051206

In [39]:
words_with_brackets = [w for w in words if '[' in w['speech_raw'] or ']' in w['speech_raw']]
len(words_with_brackets)

1002688

In [40]:
len([w for w in words if w['has_words']])

3071988

## Organizing all utterances and some summary stats

In [41]:
len(conversation_ids)

2438

In [42]:
#46.1 s on wittgenstein
utterances = []
for conversation_id in conversation_ids:
    utterances.extend(get_utterance_relations(conversation_id, 'A'))
    utterances.extend(get_utterance_relations(conversation_id, 'B'))
len(utterances)

391593

In [43]:
utterances_with_some_words = [u for u in utterances if u['has_words']]
len(utterances_with_some_words)

248826

In [44]:
def getWordformTokens(utterance_rel):
#     return utterance_rel['speech_words'].split(' ')
    return utterance_rel['speech'].split(' ')

In [45]:
wordformTokens = []
for u in utterances_with_some_words:
    wordformTokens.extend(getWordformTokens(u))
len(wordformTokens)

3071988

In [46]:
orthographic_wordform_counter_swbd = Counter(wordformTokens)
orthographic_wordform_counter_swbd.most_common()

[('i', 118103),
 ('and', 108691),
 ('the', 97331),
 ('you', 81057),
 ('a', 72629),
 ('to', 70475),
 ('uh', 69814),
 ('that', 66114),
 ('it', 55323),
 ('of', 55011),
 ('know', 48002),
 ('yeah', 47322),
 ('in', 39968),
 ('they', 32992),
 ('have', 29470),
 ('but', 28913),
 ('<rem>', 28531),
 ("it's", 27382),
 ('so', 27145),
 ('we', 25379),
 ('is', 24448),
 ('was', 23850),
 ('like', 23386),
 ('well', 22578),
 ('just', 22286),
 ('um', 21196),
 ("that's", 20848),
 ('do', 20671),
 ('for', 19691),
 ('think', 19149),
 ("don't", 18897),
 ('oh', 18825),
 ('or', 16928),
 ('on', 16749),
 ('right', 16732),
 ('uh-huh', 16294),
 ('um-hum', 16006),
 ('my', 14924),
 ('what', 14923),
 ('really', 14798),
 ('not', 14395),
 ('be', 14094),
 ('with', 13949),
 ('are', 13291),
 ('if', 13233),
 ('there', 12842),
 ('one', 12657),
 ("i'm", 12356),
 ('about', 12098),
 ('all', 11926),
 ('get', 11738),
 ('because', 11372),
 ('out', 10932),
 ('had', 10912),
 ('at', 10698),
 ('them', 10371),
 ('as', 10040),
 ('up', 982

In [47]:
orthographic_wordform_types = set(wordformTokens)
len(orthographic_wordform_types)

27559

In [48]:
hasSquareBrackets = lambda w: "[" in w or "]" in w
orthographic_wordforms_with_brackets = {w for w in orthographic_wordform_types if hasSquareBrackets(w)}
len(orthographic_wordforms_with_brackets)
orthographic_wordforms_with_brackets

0

set()

In [49]:
{w for w in orthographic_wordforms_with_brackets if w[0] == '-'}

set()

In [50]:
utts_with_brackets = [u for u in utterances_with_some_words if u['has_word_internal_brackets']]
len(utts_with_brackets)

20459

In [51]:
utts_with_brackets[:10]

[{'utterance_id': 'sw3522A-ms98-a-0006',
  'conversation_id': '3522',
  'speaker': 'A',
  'start': 24.94,
  'end': 30.14875,
  'duration': 5.208749999999998,
  'speech_raw': "see i've on[ly]- only lived here in Texas for two years so i haven't had a chance to go down there",
  'speech': "see i've <rem> only lived here in texas for two years so i haven't had a chance to go down there",
  'n_words': 21,
  'has_words': True,
  'has_word_internal_brackets': True},
 {'utterance_id': 'sw3522A-ms98-a-0024',
  'conversation_id': '3522',
  'speaker': 'A',
  'start': 98.2575,
  'end': 100.563875,
  'duration': 2.3063750000000027,
  'speech_raw': 'm[akes]- makes you feel like a kid again',
  'speech': '<rem> makes you feel like a kid again',
  'n_words': 8,
  'has_words': True,
  'has_word_internal_brackets': True},
 {'utterance_id': 'sw3522A-ms98-a-0030',
  'conversation_id': '3522',
  'speaker': 'A',
  'start': 115.567875,
  'end': 119.75425,
  'duration': 4.186374999999998,
  'speech_raw': 'y[

In [52]:
utts_with_word_initial_brackets = [u for u in utts_with_brackets 
                                   if any(map(lambda w: w[0] == '-', 
                                              u['speech_raw'].split(' ')))]
len(utts_with_word_initial_brackets)

470

In [53]:
utts_with_word_initial_brackets[:10]

[{'utterance_id': 'sw3515A-ms98-a-0013',
  'conversation_id': '3515',
  'speaker': 'A',
  'start': 58.914125,
  'end': 67.739625,
  'duration': 8.825500000000005,
  'speech_raw': "oh can you can you hold on for just a minute i'm gonna try pressing one again to make sure that i pressed it hard enough because_1 -[o]kay just a second",
  'speech': "oh can you can you hold on for just a minute i'm gonna try pressing one again to make sure that i pressed it hard enough because <rem> just a second",
  'n_words': 31,
  'has_words': True,
  'has_word_internal_brackets': True},
 {'utterance_id': 'sw3477A-ms98-a-0063',
  'conversation_id': '3477',
  'speaker': 'A',
  'start': 288.914125,
  'end': 294.12675,
  'duration': 5.212625000000003,
  'speech_raw': "-[it]'s so easy i just stick them_1 in this machine and get the number [vocalized-noise] out you know",
  'speech': '<rem> so easy i just stick them in this machine and get the number out you know',
  'n_words': 17,
  'has_words': True,
  'has

In [54]:
utts_with_word_initial_brackets

[{'utterance_id': 'sw3515A-ms98-a-0013',
  'conversation_id': '3515',
  'speaker': 'A',
  'start': 58.914125,
  'end': 67.739625,
  'duration': 8.825500000000005,
  'speech_raw': "oh can you can you hold on for just a minute i'm gonna try pressing one again to make sure that i pressed it hard enough because_1 -[o]kay just a second",
  'speech': "oh can you can you hold on for just a minute i'm gonna try pressing one again to make sure that i pressed it hard enough because <rem> just a second",
  'n_words': 31,
  'has_words': True,
  'has_word_internal_brackets': True},
 {'utterance_id': 'sw3477A-ms98-a-0063',
  'conversation_id': '3477',
  'speaker': 'A',
  'start': 288.914125,
  'end': 294.12675,
  'duration': 5.212625000000003,
  'speech_raw': "-[it]'s so easy i just stick them_1 in this machine and get the number [vocalized-noise] out you know",
  'speech': '<rem> so easy i just stick them in this machine and get the number out you know',
  'n_words': 17,
  'has_words': True,
  'has

In [55]:
utts_with_braces = [u for u in utterances_with_some_words if any(map(hasCurlyBraces, 
                                                                     u['speech_raw'].split(' ')))]
len(utts_with_braces)

352

In [56]:
utts_with_braces[:10]

[{'utterance_id': 'sw2567B-ms98-a-0087',
  'conversation_id': '2567',
  'speaker': 'B',
  'start': 366.56275,
  'end': 367.71525,
  'duration': 1.1525000000000318,
  'speech_raw': '{alrighty}',
  'speech': 'alrighty',
  'n_words': 1,
  'has_words': True,
  'has_word_internal_brackets': False},
 {'utterance_id': 'sw3017A-ms98-a-0065',
  'conversation_id': '3017',
  'speaker': 'A',
  'start': 170.61175,
  'end': 175.469125,
  'duration': 4.8573749999999905,
  'speech_raw': 'but i do they have {aquacise} which i like that quite a bit',
  'speech': 'but i do they have aquacise which i like that quite a bit',
  'n_words': 13,
  'has_words': True,
  'has_word_internal_brackets': False},
 {'utterance_id': 'sw3515A-ms98-a-0019',
  'conversation_id': '3515',
  'speaker': 'A',
  'start': 84.67325,
  'end': 93.996875,
  'duration': 9.323625000000007,
  'speech_raw': "yeah exactly and so it's like they charge you for the deposit so when you turn in the cans {unsmashed} you're just like reimbursed"

In [57]:
w_with_underscore = {w for w in orthographic_wordform_types if hasUnderscore(w)}
len(w_with_underscore)
w_with_underscore

0

set()

In [58]:
utts_with_underscore = [u for u in utterances_with_some_words if any(map(hasUnderscore, 
                                                                         u['speech'].split()))]
len(utts_with_underscore)

0

In [59]:
utts_with_underscore[:10]

[]

### Summary

There are
 - ≈2400 conversations.
 - ≈250k (pre-segmented) utterances.
 - ≈3m wordform tokens.
 - about 28k wordform types. (≥6k "orthographic wordforms" - not included in the 28k count - are interrupted or repaired wordforms.)
 
Other notes:
 - every utterance and wordform token has a duration associated with it.

# Comparison with Fisher

This assumes you have access to the Fisher corpus and have already analyzed it / processed it in some way...

In [60]:
fisher_repo_dir = '/mnt/cube/home/AD/emeinhar/fisher-lm'

In [61]:
os.chdir(fisher_repo_dir)

In [62]:
%ls *vocab*

fisher_vocabulary_bbn.txt  fisher_vocabulary_main.txt


In [63]:
%cat -n fisher_vocabulary_bbn.txt | head -20

     1	'and
     2	'berserkly'
     3	'bout
     4	'burb
     5	'burban
     6	'burbs
     7	'cau
     8	'cause
     9	'cept
    10	'cide
    11	'cisco
    12	'cize
    13	'course
    14	'cuse
    15	'do
    16	'em
    17	'em's
    18	'ems
    19	'everybody's
    20	'fess
cat: write error: Broken pipe


In [64]:
%cat -n fisher_vocabulary_main.txt | head -20

     1	'and
     2	'berserkly'
     3	'bout
     4	'burb
     5	'burban
     6	'burbs
     7	'cau
     8	'cause
     9	'cept
    10	'cide
    11	'cisco
    12	'cize
    13	'course
    14	'cuse
    15	'do
    16	'em
    17	'em's
    18	'ems
    19	'everybody's
    20	'fess
cat: write error: Broken pipe


In [65]:
fisher_vocab_bbn_fn = 'fisher_vocabulary_bbn.txt'
fisher_vocab_main_fn = 'fisher_vocabulary_main.txt'

In [66]:
fisher_vocab_bbn = []
with open(fisher_vocab_bbn_fn) as file:
    for line in file:
        fisher_vocab_bbn.append(line.rstrip())


fisher_vocab_main = []
with open(fisher_vocab_main_fn) as file:
    for line in file:
        fisher_vocab_main.append(line.rstrip())

In [67]:
len(fisher_vocab_bbn)
len(fisher_vocab_main)

42013

44064

In [68]:
fisher_vocab_bbn = set(fisher_vocab_bbn)
fisher_vocab_main = set(fisher_vocab_main)

In [69]:
def compare_sizes(before, after):
    print("{0} vs. {1}".format(len(before), len(after)))

In [70]:
def normalize_case(vocabulary):
    return set(map(lambda w: w.lower(),
                   vocabulary))

In [71]:
swbd_vocab = orthographic_wordform_types
swbd_vocab_lc = set(normalize_case(swbd_vocab))
fisher_vocab_bbn_lc = set(normalize_case(fisher_vocab_bbn))
fisher_vocab_main_lc = set(normalize_case(fisher_vocab_main))

In [72]:
compare_sizes(swbd_vocab, swbd_vocab_lc)
compare_sizes(fisher_vocab_bbn, fisher_vocab_bbn_lc)
compare_sizes(fisher_vocab_main, fisher_vocab_main_lc)

27559 vs. 27559
42013 vs. 42013
44064 vs. 44064


In [73]:
vocab_unique_to_fisher = fisher_vocab_main_lc - swbd_vocab_lc
len(vocab_unique_to_fisher)

22846

In [74]:
vocab_unique_to_swbd = swbd_vocab_lc - fisher_vocab_main_lc
len(vocab_unique_to_swbd)

6341

In [75]:
vocab_unique_to_swbd

{'amoral',
 'handcraft',
 'duncans',
 'benzes',
 'latek',
 'repeater',
 'peretti',
 'estonia',
 'roomy',
 'schematic',
 'epa',
 'dirkson',
 'covenants',
 'rejuvenator',
 'swished',
 'snider',
 'ex-wives',
 'clemson',
 'applauding',
 'fund-raising',
 'jeers',
 'raveled',
 'hallucinogen',
 "swayze's",
 'twirled',
 'schoolyard',
 'showbusiness',
 'vacationer',
 'holy-moly',
 'backlogged',
 'fervent',
 'reinjuring',
 'malts',
 'monastery',
 'interfaces',
 'dx',
 '2ci',
 'compressors',
 'mugging',
 "hodgkin's",
 "evaluation's",
 'unseasonal',
 'personably',
 'aptly',
 'bitterest',
 'chagrin',
 'wesson',
 "custer's",
 're-texture',
 'knifings',
 'marimba',
 'judiciary',
 'wallowed',
 'reteaching',
 'noontime',
 "ellsworth's",
 'nonsectarian',
 'unix',
 'sedonia',
 'sparkplugs',
 'eec',
 'mastercards',
 'buttercup',
 'delancy',
 'chairmen',
 'mancha',
 'expatriates',
 'self-control',
 'centerville',
 'chapman',
 'reiner',
 'tdy',
 'roby',
 'maroon',
 'diskettes',
 'mollycoddle',
 'anomalies',

In [76]:
swbd_unique_vocab_by_freq = Counter({w:orthographic_wordform_counter_swbd[w] for w in vocab_unique_to_swbd})
swbd_unique_vocab_by_freq.most_common()

[('ibm', 117),
 ('hum-um', 96),
 ('bins', 85),
 ('hmo', 66),
 ('recyclable', 48),
 ('vcr', 45),
 ('play-offs', 44),
 ('lewisville', 40),
 ('freon', 38),
 ('nra', 31),
 ('pbs', 30),
 ('gm', 29),
 ('nfl', 27),
 ('addison', 27),
 ('byu', 26),
 ('cpa', 26),
 ('switchboard', 26),
 ('nc', 23),
 ("ti's", 23),
 ('dukakis', 23),
 ('unix', 22),
 ('nba', 22),
 ('amiga', 22),
 ('oldsmobile', 22),
 ('nolan', 22),
 ('aetna', 22),
 ('recyclables', 22),
 ('fajitas', 21),
 ('clarion', 21),
 ('eds', 21),
 ('republics', 20),
 ('tex-mex', 20),
 ('tarpley', 20),
 ('epa', 18),
 ("cd's", 18),
 ('bmw', 18),
 ('irs', 18),
 ('tvs', 18),
 ('molding', 17),
 ('muffler', 17),
 ('fords', 17),
 ('aikman', 17),
 ('denton', 17),
 ('rowlett', 16),
 ('attleboro', 16),
 ('gasket', 16),
 ('smu', 16),
 ('coda', 16),
 ('yeltsin', 16),
 ('flatliners', 16),
 ('framing', 16),
 ('tole', 15),
 ('utd', 15),
 ('illustrated', 15),
 ('doctoral', 14),
 ('semiautomatic', 14),
 ('unlv', 14),
 ('20/20', 14),
 ('subaru', 14),
 ('crappie',

In [77]:
swbd_wfs_w_braces = {w for w in vocab_unique_to_swbd if '{' in w}
len(swbd_wfs_w_braces)
swbd_wfs_w_braces

0

set()

In [78]:
[w for w in vocab_unique_to_swbd if hasCurlyBraces(w) and not isCurlyBraced(w)]

[]

# Relating the word and utterance relations

In [79]:
utterances[10]
utterances[11]

{'utterance_id': 'sw3522A-ms98-a-0011',
 'conversation_id': '3522',
 'speaker': 'A',
 'start': 41.196,
 'end': 42.8735,
 'duration': 1.677500000000002,
 'speech_raw': 'yeah',
 'speech': 'yeah',
 'n_words': 1,
 'has_words': True,
 'has_word_internal_brackets': False}

{'utterance_id': 'sw3522A-ms98-a-0012',
 'conversation_id': '3522',
 'speaker': 'A',
 'start': 42.8735,
 'end': 44.278,
 'duration': 1.4044999999999987,
 'speech_raw': 'well',
 'speech': 'well',
 'n_words': 1,
 'has_words': True,
 'has_word_internal_brackets': False}

In [80]:
[u for u in utterances if u['utterance_id'] == 'sw2923A-ms98-a-0012']

[{'utterance_id': 'sw2923A-ms98-a-0012',
  'conversation_id': '2923',
  'speaker': 'A',
  'start': 44.795875,
  'end': 49.45625,
  'duration': 4.660374999999995,
  'speech_raw': "it's just music by different groups it's you know things you you wouldn't you wouldn't uh",
  'speech': "it's just music by different groups it's you know things you you wouldn't you wouldn't uh",
  'n_words': 16,
  'has_words': True,
  'has_word_internal_brackets': False}]

## Mapping each utterance ID to a sequence of word relations

Given an utterance id, we want to be able to retrieve the corresponding sequence of elements in the word relation.

In [81]:
def get_word_relation_seq(utterance_id, remove_non_speech=True):
    w_rels = [w for w in words if w['utterance_id'] == utterance_id]
    if remove_non_speech:
        w_rels = [w for w in w_rels if w['has_words']]
    w_rels_sorted = sorted(w_rels, key=lambda w_rel: w_rel['start'])
    return w_rels

In [82]:
[u for u in utterances if u['utterance_id'] == 'sw2923A-ms98-a-0012']

[{'utterance_id': 'sw2923A-ms98-a-0012',
  'conversation_id': '2923',
  'speaker': 'A',
  'start': 44.795875,
  'end': 49.45625,
  'duration': 4.660374999999995,
  'speech_raw': "it's just music by different groups it's you know things you you wouldn't you wouldn't uh",
  'speech': "it's just music by different groups it's you know things you you wouldn't you wouldn't uh",
  'n_words': 16,
  'has_words': True,
  'has_word_internal_brackets': False}]

In [83]:
words_in_the_utt = get_word_relation_seq('sw2923A-ms98-a-0012', False)
len(words_in_the_utt)
only_words = [w for w in words_in_the_utt if w['has_words']]
len(only_words)
' '.join([w['speech'] for w in only_words])
' '.join([w['speech'] for w in only_words]) == [u for u in utterances if u['utterance_id'] == 'sw2923A-ms98-a-0012'][0]['speech']

18

16

"it's just music by different groups it's you know things you you wouldn't you wouldn't uh"

True

In [84]:
get_word_relation_seq('sw2923A-ms98-a-0012', True)

[{'utterance_id': 'sw2923A-ms98-a-0012',
  'conversation_id': '2923',
  'speaker': 'A',
  'start': 45.565875,
  'end': 45.725875,
  'duration': 0.1600000000000037,
  'speech_raw': "it's",
  'speech': "it's",
  'has_words': True,
  'has_word_internal_brackets': False},
 {'utterance_id': 'sw2923A-ms98-a-0012',
  'conversation_id': '2923',
  'speaker': 'A',
  'start': 45.725875,
  'end': 46.105875,
  'duration': 0.37999999999999545,
  'speech_raw': 'just',
  'speech': 'just',
  'has_words': True,
  'has_word_internal_brackets': False},
 {'utterance_id': 'sw2923A-ms98-a-0012',
  'conversation_id': '2923',
  'speaker': 'A',
  'start': 46.105875,
  'end': 46.375875,
  'duration': 0.2700000000000031,
  'speech_raw': 'music',
  'speech': 'music',
  'has_words': True,
  'has_word_internal_brackets': False},
 {'utterance_id': 'sw2923A-ms98-a-0012',
  'conversation_id': '2923',
  'speaker': 'A',
  'start': 46.375875,
  'end': 46.475875,
  'duration': 0.10000000000000142,
  'speech_raw': 'by',
  '

## Mapping each utterance ID to an orthographic word sequence + word duration sequence pair

In [85]:
def get_words_and_durations(utterance_id):
    w_rels = get_word_relation_seq(utterance_id)
    orth_wfs = tuple([w['speech'] for w in w_rels])
    durations = tuple([w['duration'] for w in w_rels])
#     assert len(orth_wfs) == len(durations)
    return orth_wfs, durations

In [86]:
get_words_and_durations('sw2923A-ms98-a-0012')

(("it's",
  'just',
  'music',
  'by',
  'different',
  'groups',
  "it's",
  'you',
  'know',
  'things',
  'you',
  'you',
  "wouldn't",
  'you',
  "wouldn't",
  'uh'),
 (0.1600000000000037,
  0.37999999999999545,
  0.2700000000000031,
  0.10000000000000142,
  0.3200000000000003,
  0.3499999999999943,
  0.20000000000000284,
  0.060000000000002274,
  0.22999999999999687,
  0.18312500000000398,
  0.13787499999999397,
  0.08900000000000574,
  0.17999999999999972,
  0.0899999999999963,
  0.3100000000000023,
  0.1700000000000017))

In [87]:
tuple(zip(*get_words_and_durations('sw2923A-ms98-a-0012')))

(("it's", 0.1600000000000037),
 ('just', 0.37999999999999545),
 ('music', 0.2700000000000031),
 ('by', 0.10000000000000142),
 ('different', 0.3200000000000003),
 ('groups', 0.3499999999999943),
 ("it's", 0.20000000000000284),
 ('you', 0.060000000000002274),
 ('know', 0.22999999999999687),
 ('things', 0.18312500000000398),
 ('you', 0.13787499999999397),
 ('you', 0.08900000000000574),
 ("wouldn't", 0.17999999999999972),
 ('you', 0.0899999999999963),
 ("wouldn't", 0.3100000000000023),
 ('uh', 0.1700000000000017))

## Making a word relation for confusability analysis

For each word in the corpus, we want
 - the orthographic wordform
 - all preceding wordforms in the utterance
 - the 1,2,3,4 preceding wordforms in the utterance
 - all following wordforms in the utterance
 - the 1,2,3,4 following wordforms in the utterance
 - whether the word is adjacent to any disfluencies, pauses, or filled pauses
 - whether the word begins or ends with a clitic
 - the duration
 - the utterance ID
 - the conversation ID
 - the speaker

In [88]:
def isPauseOrDisfluency(speech_word_token):
    return speech_word_token == unk
#     if type(container) == b.containers.Word:
#         return False
#     return True


filledPauses = {
 'ah',
 'hm',
 'hmm',
#  'huh',
#  'huh-uh',
#  'hum',
#  'hum-um',
#  'mm',
#  'mm-hmm',
 'uh',
#  'uh-huh',
#  'uh-hum',
#  'uh-uh',
 'um',
#  'um-hmm',
#  'um-huh',
#  'um-hum'
}

def isFilledPause(speech_word_token):
    return speech_word_token in filledPauses

In [89]:
vocabulary = orthographic_wordform_types

In [90]:
"yknow" in vocabulary
"y'know" in vocabulary
"ya" in vocabulary

False

False

False

In [91]:
clitic_suffixes = {"n't", "'ve", "'ll", "'s", "s'","'d", "'re", "'m", "'em", "'d've"}

In [92]:
words_with_clitic_suffix = {w for w in vocabulary 
                            if any({suff in w for suff in clitic_suffixes})}
len(vocabulary)
len(words_with_clitic_suffix)
len(words_with_clitic_suffix - {w for w in words_with_clitic_suffix
                                if "'s" in w})

27559

1350

169

In [93]:
words_with_clitic_suffix - {w for w in words_with_clitic_suffix
                                if "'s" in w or "s'" in w}

{"ain't",
 "aren't",
 "ashley'll",
 "ball'll",
 "baltimore'll",
 "brother'll",
 "can't",
 "can'ts",
 "car'll",
 "cat'll",
 "children'll",
 "color'd",
 "could've",
 "couldn't",
 "day'll",
 "didn't",
 "doesn't",
 "don't",
 "don'ts",
 "duke'll",
 "everybody'd",
 "grandmother'll",
 "hadn't",
 "hasn't",
 "haven't",
 "he'd",
 "he'll",
 "how'd",
 "husband'll",
 "i'd",
 "i'll",
 "i'm",
 "i've",
 "isn't",
 "it'd",
 "it'll",
 "kid'll",
 "lady'll",
 "market'll",
 "may've",
 "might've",
 "mine'd",
 "must've",
 "needn't",
 "neighbor'll",
 "nobody'll",
 "nothing'll",
 "one'll",
 "people'll",
 "physician'll",
 "rest'll",
 "shan't",
 "she'd",
 "she'll",
 "should've",
 "shouldn't",
 "shouldn'ts",
 "some'll",
 "somebody'd",
 "somebody'll",
 "someone'll",
 "that'd",
 "that'll",
 "there'd",
 "there'll",
 "there're",
 "there've",
 "they'd",
 "they'll",
 "they're",
 "they've",
 "to've",
 "town'd",
 "tracy'd",
 "valentine'll",
 "wasn't",
 "we'd",
 "we'll",
 "we're",
 "we've",
 "weren't",
 "what'd",
 "what'll

In [94]:
def hasClitic(orthographic_wordform_token):
    w = orthographic_wordform_token
    return w in words_with_clitic_suffix

In [95]:
s = "it's just music by different groups it's you know things you you wouldn't you wouldn't uh"
ws = s.split(' ')
len(ws)
i = 3
j = 8

ws[j]
ws[j-4 if j-4 > 0 else 0:j]

ws[i]
ws[i-4 if i-4 > 0 else 0:i]

def get_ngram_context(word_seq, i, n=None):
    if n is None:
        n = 4
    assert n > 0
    assert i > -1
    assert i < len(word_seq)
    return word_seq[i-n if i-n > 0 else 0:i]

get_ngram_context(ws, 8)
get_ngram_context(ws, 3)

def get_ngram_contexts(word_seq, n=None):
    if n is None:
        n = 4
    assert n > 0
    return tuple(map(lambda i: get_ngram_context(word_seq, i),
                     range(len(word_seq))))

def get_all_prefixes(word_seq):
    return [word_seq[0:i] for i in range(len(word_seq)+1)]

def get_all_contexts(word_seq):
    return get_all_prefixes(word_seq)[:-1]

get_ngram_contexts(ws)
get_all_contexts(ws)

16

'know'

['different', 'groups', "it's", 'you']

'by'

["it's", 'just', 'music']

['different', 'groups', "it's", 'you']

["it's", 'just', 'music']

([],
 ["it's"],
 ["it's", 'just'],
 ["it's", 'just', 'music'],
 ["it's", 'just', 'music', 'by'],
 ['just', 'music', 'by', 'different'],
 ['music', 'by', 'different', 'groups'],
 ['by', 'different', 'groups', "it's"],
 ['different', 'groups', "it's", 'you'],
 ['groups', "it's", 'you', 'know'],
 ["it's", 'you', 'know', 'things'],
 ['you', 'know', 'things', 'you'],
 ['know', 'things', 'you', 'you'],
 ['things', 'you', 'you', "wouldn't"],
 ['you', 'you', "wouldn't", 'you'],
 ['you', "wouldn't", 'you', "wouldn't"])

[[],
 ["it's"],
 ["it's", 'just'],
 ["it's", 'just', 'music'],
 ["it's", 'just', 'music', 'by'],
 ["it's", 'just', 'music', 'by', 'different'],
 ["it's", 'just', 'music', 'by', 'different', 'groups'],
 ["it's", 'just', 'music', 'by', 'different', 'groups', "it's"],
 ["it's", 'just', 'music', 'by', 'different', 'groups', "it's", 'you'],
 ["it's", 'just', 'music', 'by', 'different', 'groups', "it's", 'you', 'know'],
 ["it's",
  'just',
  'music',
  'by',
  'different',
  'groups',
  "it's",
  'you',
  'know',
  'things'],
 ["it's",
  'just',
  'music',
  'by',
  'different',
  'groups',
  "it's",
  'you',
  'know',
  'things',
  'you'],
 ["it's",
  'just',
  'music',
  'by',
  'different',
  'groups',
  "it's",
  'you',
  'know',
  'things',
  'you',
  'you'],
 ["it's",
  'just',
  'music',
  'by',
  'different',
  'groups',
  "it's",
  'you',
  'know',
  'things',
  'you',
  'you',
  "wouldn't"],
 ["it's",
  'just',
  'music',
  'by',
  'different',
  'groups',
  "it's",
  'you',
  'kno

In [96]:
def get_words_durations_and_contexts(utterance_id, n=None):
    if n is None:
        n = 4
    
    w_rels = get_word_relation_seq(utterance_id)
    orth_wfs = tuple([w['speech'] for w in w_rels])
    durations = tuple([w['duration'] for w in w_rels])
    local_contexts = get_ngram_contexts(orth_wfs)
    full_contexts = get_all_contexts(orth_wfs)
#     assert len(orth_wfs) == len(durations)
    return orth_wfs, durations, local_contexts, full_contexts

In [97]:
get_words_durations_and_contexts('sw2923A-ms98-a-0012')

(("it's",
  'just',
  'music',
  'by',
  'different',
  'groups',
  "it's",
  'you',
  'know',
  'things',
  'you',
  'you',
  "wouldn't",
  'you',
  "wouldn't",
  'uh'),
 (0.1600000000000037,
  0.37999999999999545,
  0.2700000000000031,
  0.10000000000000142,
  0.3200000000000003,
  0.3499999999999943,
  0.20000000000000284,
  0.060000000000002274,
  0.22999999999999687,
  0.18312500000000398,
  0.13787499999999397,
  0.08900000000000574,
  0.17999999999999972,
  0.0899999999999963,
  0.3100000000000023,
  0.1700000000000017),
 ((),
  ("it's",),
  ("it's", 'just'),
  ("it's", 'just', 'music'),
  ("it's", 'just', 'music', 'by'),
  ('just', 'music', 'by', 'different'),
  ('music', 'by', 'different', 'groups'),
  ('by', 'different', 'groups', "it's"),
  ('different', 'groups', "it's", 'you'),
  ('groups', "it's", 'you', 'know'),
  ("it's", 'you', 'know', 'things'),
  ('you', 'know', 'things', 'you'),
  ('know', 'things', 'you', 'you'),
  ('things', 'you', 'you', "wouldn't"),
  ('you', 'y

In [98]:
tuple(zip(*get_words_durations_and_contexts('sw2923A-ms98-a-0012')))

(("it's", 0.1600000000000037, (), ()),
 ('just', 0.37999999999999545, ("it's",), ("it's",)),
 ('music', 0.2700000000000031, ("it's", 'just'), ("it's", 'just')),
 ('by',
  0.10000000000000142,
  ("it's", 'just', 'music'),
  ("it's", 'just', 'music')),
 ('different',
  0.3200000000000003,
  ("it's", 'just', 'music', 'by'),
  ("it's", 'just', 'music', 'by')),
 ('groups',
  0.3499999999999943,
  ('just', 'music', 'by', 'different'),
  ("it's", 'just', 'music', 'by', 'different')),
 ("it's",
  0.20000000000000284,
  ('music', 'by', 'different', 'groups'),
  ("it's", 'just', 'music', 'by', 'different', 'groups')),
 ('you',
  0.060000000000002274,
  ('by', 'different', 'groups', "it's"),
  ("it's", 'just', 'music', 'by', 'different', 'groups', "it's")),
 ('know',
  0.22999999999999687,
  ('different', 'groups', "it's", 'you'),
  ("it's", 'just', 'music', 'by', 'different', 'groups', "it's", 'you')),
 ('things',
  0.18312500000000398,
  ('groups', "it's", 'you', 'know'),
  ("it's",
   'just',


In [99]:
[u for u in utterances if u['utterance_id'] == 'sw2923A-ms98-a-0012']

[{'utterance_id': 'sw2923A-ms98-a-0012',
  'conversation_id': '2923',
  'speaker': 'A',
  'start': 44.795875,
  'end': 49.45625,
  'duration': 4.660374999999995,
  'speech_raw': "it's just music by different groups it's you know things you you wouldn't you wouldn't uh",
  'speech': "it's just music by different groups it's you know things you you wouldn't you wouldn't uh",
  'n_words': 16,
  'has_words': True,
  'has_word_internal_brackets': False}]

In [100]:
len(ws)
ws[-1]
ws[len(ws)-1]
((len(ws)-15)-1)
ws[15]
' '
ws[0]
ws[len(ws)-len(ws)]

ws[3]
(len(ws) - 3)
len(ws) - (len(ws) - 3)
ws[len(ws) - (len(ws) - 3)]
ws[-13]

16

'uh'

'uh'

0

'uh'

...

'by'

13

3

'by'

'by'

In [ ]:
utterance_ids = [u['utterance_id'] for u in utterances]
len(utterance_ids)

In [133]:
utterances_sorted = sorted(utterances, key=lambda utt_rel: utt_rel['utterance_id'])
utterance_ids_sorted = sorted(utterance_ids)

len(utterance_ids)
len(utterances_sorted)

assert all(utterance_ids_sorted[i] == utterances_sorted[i]['utterance_id'] 
           for i in tqdm(range(len(utterance_ids))))

391593

391593

100%|██████████| 391593/391593 [00:00<00:00, 1375342.45it/s]


In [135]:
# utterance_id_to_utt_rel = {utterance_id:utterances_sorted[utterance_ids_sorted.index(utterance_id)]
#                            for utterance_id in tqdm(utterance_ids)}
# assert all(len(utterance_id_to_utt_rel[utterance_id]) == 1 for utterance_id in utterance_ids)
# utterance_id_to_utt_rel = {utterance_id:utterance_id_to_utt_rel[utterance_id][0]
#                            for utterance_id in tqdm(utterance_ids)}

  1%|          | 2625/391593 [00:40<1:41:06, 64.12it/s]

In [136]:
def hasAdjacentPauseOrDisfluency(left_rel, right_rel):
    if left_rel is not None:
        return isPauseOrDisfluency(left_rel['orthographic_wordform'])
    if right_rel is not None:
        return isPauseOrDisfluency(right_rel['orthographic_wordform'])
    return False

def hasAdjacentFilledPause(left_rel, right_rel):
    if left_rel is not None:
        return isFilledPause(left_rel['orthographic_wordform'])
    if right_rel is not None:
        return isFilledPause(right_rel['orthographic_wordform'])
    return False

def make_analysis_relations(utterance_id):
#     utt_rel_matches = [u for u in utterances if u['utterance_id'] == utterance_id]
#     assert len(utt_rel_matches) > 0, 'No matching utterance found for id {0}'.format(utterance_id)
#     assert len(utt_rel_matches) == 1, 'Should be exactly one match for an utterance id, got these instead\n{0}'.format(utt_rel_matches)
#     utt_rel = utt_rel_matches[0]
    utt_rel = utterances_sorted[utterance_ids_sorted.index(utterance_id)]
    
    orth_wfs, durations, local_contexts, full_contexts = get_words_durations_and_contexts(utterance_id)
    zipped = tuple(zip(orth_wfs, durations, local_contexts, full_contexts))
    make_rel = lambda triple: {'utterance_id':utterance_id,
                               'conversation_id':utt_rel['conversation_id'],
                               'speaker':utt_rel['speaker'],
                               'orthographic_wordform':triple[0],
                               'duration':triple[1],
                               'preceding_4_wordforms':triple[2],
                               'preceding_3_wordforms':triple[2][1:],
                               'preceding_2_wordforms':triple[2][2:],
                               'preceding_2_wordforms':triple[2][3:],
                               'preceding_1_wordforms':triple[2][4:],
#                                'Preceding_4_Wordforms':list(triple[2]),
                               'preceding_wordforms':triple[3]
                              #FIXME add following contexts
                              }
    as_rels = list(map(make_rel, zipped))
    for i, w_rel in enumerate(as_rels):
        w_rel.update({'dist_from_left_edge':i})
        w_rel.update({'dist_from_right_edge':len(as_rels)-i-1})
        
        if i > 0:
            left_rel = as_rels[i-1]
        else:
            left_rel = None
        if i < len(as_rels) - 2:
            right_rel = as_rels[i+1]
        else:
            right_rel = None
        w_rel.update({'hasAdjacentPauseOrDisfluency':hasAdjacentPauseOrDisfluency(left_rel, right_rel)})
        w_rel.update({'hasAdjacentFilledPause':hasAdjacentFilledPause(left_rel, right_rel)})
        
        w_rel.update({'hasClitic':hasClitic(w_rel['orthographic_wordform'])})
        
    return as_rels

make_analysis_relations('sw2923A-ms98-a-0012')

[{'utterance_id': 'sw2923A-ms98-a-0012',
  'conversation_id': '2923',
  'speaker': 'A',
  'orthographic_wordform': "it's",
  'duration': 0.1600000000000037,
  'preceding_4_wordforms': (),
  'preceding_3_wordforms': (),
  'preceding_wordforms': (),
  'dist_from_left_edge': 0,
  'dist_from_right_edge': 15,
  'hasAdjacentPauseOrDisfluency': False,
  'hasAdjacentFilledPause': False,
  'hasClitic': True},
 {'utterance_id': 'sw2923A-ms98-a-0012',
  'conversation_id': '2923',
  'speaker': 'A',
  'orthographic_wordform': 'just',
  'duration': 0.37999999999999545,
  'preceding_4_wordforms': ("it's",),
  'preceding_3_wordforms': (),
  'preceding_wordforms': ("it's",),
  'dist_from_left_edge': 1,
  'dist_from_right_edge': 14,
  'hasAdjacentPauseOrDisfluency': False,
  'hasAdjacentFilledPause': False,
  'hasClitic': False},
 {'utterance_id': 'sw2923A-ms98-a-0012',
  'conversation_id': '2923',
  'speaker': 'A',
  'orthographic_wordform': 'music',
  'duration': 0.2700000000000031,
  'preceding_4_wor

In [102]:
# utterance_ids = [u['utterance_id'] for u in utterances]
# len(utterance_ids)

391593

In [103]:
V = 10

In [104]:
# analysis_relations = list(map(make_analysis_relations,
#                               utterance_ids))

#used to take 305m on wittgenstein with J = 30; can take around that amount of time currently
# with extra exclusion annotations and stuff going on in the background, this can now take up to ~355m
analysis_relations = par(delayed(make_analysis_relations)(utterance_id) for utterance_id in utterance_ids)
len(analysis_relations)

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done  49 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 117 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:   14.2s
[Parallel(n_jobs=-1)]: Done 178 tasks      | elapsed:   16.0s
[Parallel(n_jobs=-1)]: Done 201 tasks      | elapsed:   17.6s
[Parallel(n_jobs=-1)]: Done 224 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-1)]: Done 249 tasks      | elapsed:   21.8s
[Parallel(n_jobs=-1)]: Done 274 tasks      |

[Parallel(n_jobs=-1)]: Done 10018 tasks      | elapsed: 13.9min
[Parallel(n_jobs=-1)]: Done 10161 tasks      | elapsed: 14.1min
[Parallel(n_jobs=-1)]: Done 10304 tasks      | elapsed: 14.3min
[Parallel(n_jobs=-1)]: Done 10449 tasks      | elapsed: 14.5min
[Parallel(n_jobs=-1)]: Done 10594 tasks      | elapsed: 14.7min
[Parallel(n_jobs=-1)]: Done 10741 tasks      | elapsed: 14.9min
[Parallel(n_jobs=-1)]: Done 10888 tasks      | elapsed: 15.0min
[Parallel(n_jobs=-1)]: Done 11037 tasks      | elapsed: 15.2min
[Parallel(n_jobs=-1)]: Done 11186 tasks      | elapsed: 15.4min
[Parallel(n_jobs=-1)]: Done 11337 tasks      | elapsed: 15.7min
[Parallel(n_jobs=-1)]: Done 11488 tasks      | elapsed: 15.9min
[Parallel(n_jobs=-1)]: Done 11641 tasks      | elapsed: 16.1min
[Parallel(n_jobs=-1)]: Done 11794 tasks      | elapsed: 16.3min
[Parallel(n_jobs=-1)]: Done 11949 tasks      | elapsed: 16.5min
[Parallel(n_jobs=-1)]: Done 12104 tasks      | elapsed: 16.7min
[Parallel(n_jobs=-1)]: Done 12261 tasks 

[Parallel(n_jobs=-1)]: Done 36657 tasks      | elapsed: 50.7min
[Parallel(n_jobs=-1)]: Done 36928 tasks      | elapsed: 51.1min
[Parallel(n_jobs=-1)]: Done 37201 tasks      | elapsed: 51.4min
[Parallel(n_jobs=-1)]: Done 37474 tasks      | elapsed: 51.8min
[Parallel(n_jobs=-1)]: Done 37749 tasks      | elapsed: 52.2min
[Parallel(n_jobs=-1)]: Done 38024 tasks      | elapsed: 52.5min
[Parallel(n_jobs=-1)]: Done 38301 tasks      | elapsed: 52.9min
[Parallel(n_jobs=-1)]: Done 38578 tasks      | elapsed: 53.3min
[Parallel(n_jobs=-1)]: Done 38857 tasks      | elapsed: 53.7min
[Parallel(n_jobs=-1)]: Done 39136 tasks      | elapsed: 54.1min
[Parallel(n_jobs=-1)]: Done 39417 tasks      | elapsed: 54.5min
[Parallel(n_jobs=-1)]: Done 39698 tasks      | elapsed: 54.9min
[Parallel(n_jobs=-1)]: Done 39981 tasks      | elapsed: 55.3min
[Parallel(n_jobs=-1)]: Done 40264 tasks      | elapsed: 55.7min
[Parallel(n_jobs=-1)]: Done 40549 tasks      | elapsed: 56.1min
[Parallel(n_jobs=-1)]: Done 40834 tasks 

[Parallel(n_jobs=-1)]: Done 79537 tasks      | elapsed: 109.1min
[Parallel(n_jobs=-1)]: Done 79936 tasks      | elapsed: 109.6min
[Parallel(n_jobs=-1)]: Done 80337 tasks      | elapsed: 110.1min
[Parallel(n_jobs=-1)]: Done 80738 tasks      | elapsed: 110.7min
[Parallel(n_jobs=-1)]: Done 81141 tasks      | elapsed: 111.2min
[Parallel(n_jobs=-1)]: Done 81544 tasks      | elapsed: 111.8min
[Parallel(n_jobs=-1)]: Done 81949 tasks      | elapsed: 112.4min
[Parallel(n_jobs=-1)]: Done 82354 tasks      | elapsed: 112.9min
[Parallel(n_jobs=-1)]: Done 82761 tasks      | elapsed: 113.5min
[Parallel(n_jobs=-1)]: Done 83168 tasks      | elapsed: 114.0min
[Parallel(n_jobs=-1)]: Done 83577 tasks      | elapsed: 114.5min
[Parallel(n_jobs=-1)]: Done 83986 tasks      | elapsed: 115.1min
[Parallel(n_jobs=-1)]: Done 84397 tasks      | elapsed: 115.6min
[Parallel(n_jobs=-1)]: Done 84808 tasks      | elapsed: 116.2min
[Parallel(n_jobs=-1)]: Done 85221 tasks      | elapsed: 116.8min
[Parallel(n_jobs=-1)]: Do

[Parallel(n_jobs=-1)]: Done 137224 tasks      | elapsed: 187.7min
[Parallel(n_jobs=-1)]: Done 137749 tasks      | elapsed: 188.4min
[Parallel(n_jobs=-1)]: Done 138274 tasks      | elapsed: 189.1min
[Parallel(n_jobs=-1)]: Done 138801 tasks      | elapsed: 189.9min
[Parallel(n_jobs=-1)]: Done 139328 tasks      | elapsed: 190.6min
[Parallel(n_jobs=-1)]: Done 139857 tasks      | elapsed: 191.3min
[Parallel(n_jobs=-1)]: Done 140386 tasks      | elapsed: 192.1min
[Parallel(n_jobs=-1)]: Done 140917 tasks      | elapsed: 192.8min
[Parallel(n_jobs=-1)]: Done 141448 tasks      | elapsed: 193.5min
[Parallel(n_jobs=-1)]: Done 141981 tasks      | elapsed: 194.3min
[Parallel(n_jobs=-1)]: Done 142514 tasks      | elapsed: 195.0min
[Parallel(n_jobs=-1)]: Done 143049 tasks      | elapsed: 195.7min
[Parallel(n_jobs=-1)]: Done 143584 tasks      | elapsed: 196.4min
[Parallel(n_jobs=-1)]: Done 144121 tasks      | elapsed: 197.2min
[Parallel(n_jobs=-1)]: Done 144658 tasks      | elapsed: 197.9min
[Parallel(

[Parallel(n_jobs=-1)]: Done 210537 tasks      | elapsed: 287.4min
[Parallel(n_jobs=-1)]: Done 211186 tasks      | elapsed: 288.3min
[Parallel(n_jobs=-1)]: Done 211837 tasks      | elapsed: 289.1min
[Parallel(n_jobs=-1)]: Done 212488 tasks      | elapsed: 290.1min
[Parallel(n_jobs=-1)]: Done 213141 tasks      | elapsed: 290.9min
[Parallel(n_jobs=-1)]: Done 213794 tasks      | elapsed: 291.8min
[Parallel(n_jobs=-1)]: Done 214449 tasks      | elapsed: 292.7min
[Parallel(n_jobs=-1)]: Done 215104 tasks      | elapsed: 293.6min
[Parallel(n_jobs=-1)]: Done 215761 tasks      | elapsed: 294.5min
[Parallel(n_jobs=-1)]: Done 216418 tasks      | elapsed: 295.4min
[Parallel(n_jobs=-1)]: Done 217077 tasks      | elapsed: 296.3min
[Parallel(n_jobs=-1)]: Done 217736 tasks      | elapsed: 297.2min
[Parallel(n_jobs=-1)]: Done 218397 tasks      | elapsed: 298.1min
[Parallel(n_jobs=-1)]: Done 219058 tasks      | elapsed: 299.0min
[Parallel(n_jobs=-1)]: Done 219721 tasks      | elapsed: 299.9min
[Parallel(

[Parallel(n_jobs=-1)]: Done 299474 tasks      | elapsed: 408.2min
[Parallel(n_jobs=-1)]: Done 300249 tasks      | elapsed: 409.3min
[Parallel(n_jobs=-1)]: Done 301024 tasks      | elapsed: 410.4min
[Parallel(n_jobs=-1)]: Done 301801 tasks      | elapsed: 411.5min
[Parallel(n_jobs=-1)]: Done 302578 tasks      | elapsed: 412.5min
[Parallel(n_jobs=-1)]: Done 303357 tasks      | elapsed: 413.5min
[Parallel(n_jobs=-1)]: Done 304136 tasks      | elapsed: 414.6min
[Parallel(n_jobs=-1)]: Done 304917 tasks      | elapsed: 415.7min
[Parallel(n_jobs=-1)]: Done 305698 tasks      | elapsed: 416.7min
[Parallel(n_jobs=-1)]: Done 306481 tasks      | elapsed: 417.8min
[Parallel(n_jobs=-1)]: Done 307264 tasks      | elapsed: 418.9min
[Parallel(n_jobs=-1)]: Done 308049 tasks      | elapsed: 420.0min
[Parallel(n_jobs=-1)]: Done 308834 tasks      | elapsed: 421.0min
[Parallel(n_jobs=-1)]: Done 309621 tasks      | elapsed: 422.1min
[Parallel(n_jobs=-1)]: Done 310408 tasks      | elapsed: 423.2min
[Parallel(

391593

In [105]:
word_analysis_relation = list(chain.from_iterable(analysis_relations))
len(word_analysis_relation)

3071988

In [106]:
word_analysis_relation[0]

{'utterance_id': 'sw3522A-ms98-a-0003',
 'conversation_id': '3522',
 'speaker': 'A',
 'orthographic_wordform': 'where',
 'duration': 0.18262499999999993,
 'preceding_4_wordforms': (),
 'preceding_3_wordforms': (),
 'preceding_wordforms': (),
 'dist_from_left_edge': 0,
 'dist_from_right_edge': 7,
 'hasAdjacentPauseOrDisfluency': False,
 'hasAdjacentFilledPause': False,
 'hasClitic': False}

In [107]:
[r for r in word_analysis_relation if r['orthographic_wordform'] == '<rem>']

[{'utterance_id': 'sw3522A-ms98-a-0006',
  'conversation_id': '3522',
  'speaker': 'A',
  'orthographic_wordform': '<rem>',
  'duration': 0.28999999999999915,
  'preceding_4_wordforms': ('see', "i've"),
  'preceding_3_wordforms': ("i've",),
  'preceding_wordforms': ('see', "i've"),
  'dist_from_left_edge': 2,
  'dist_from_right_edge': 18,
  'hasAdjacentPauseOrDisfluency': False,
  'hasAdjacentFilledPause': False,
  'hasClitic': False},
 {'utterance_id': 'sw3522A-ms98-a-0024',
  'conversation_id': '3522',
  'speaker': 'A',
  'orthographic_wordform': '<rem>',
  'duration': 0.030000000000001137,
  'preceding_4_wordforms': (),
  'preceding_3_wordforms': (),
  'preceding_wordforms': (),
  'dist_from_left_edge': 0,
  'dist_from_right_edge': 7,
  'hasAdjacentPauseOrDisfluency': False,
  'hasAdjacentFilledPause': False,
  'hasClitic': False},
 {'utterance_id': 'sw3522A-ms98-a-0030',
  'conversation_id': '3522',
  'speaker': 'A',
  'orthographic_wordform': '<rem>',
  'duration': 0.2321249999999

## Identifying n-gram contexts

In [108]:
local_contexts = [r['preceding_4_wordforms'] for r in word_analysis_relation]
len(local_contexts)

3071988

In [109]:
contexts_fourgram = [tuple(c) for c in local_contexts if len(c) == 4]
len(contexts_fourgram)

2314779

In [110]:
Counter(contexts_fourgram).most_common()

[(('i', "don't", 'know', 'if'), 757),
 (('i', "don't", 'know', 'i'), 729),
 (('a', 'lot', 'of', 'people'), 608),
 (('a', 'lot', 'of', 'the'), 493),
 (('i', "don't", 'know', 'what'), 457),
 (("there's", 'a', 'lot', 'of'), 402),
 (('have', 'a', 'lot', 'of'), 370),
 (('and', 'uh', 'you', 'know'), 368),
 (('and', 'things', 'like', 'that'), 355),
 (('i', "don't", 'i', "don't"), 353),
 (('uh', 'i', "don't", 'know'), 343),
 (('i', "don't", 'know', 'how'), 338),
 (('what', 'do', 'you', 'think'), 335),
 (('or', 'something', 'like', 'that'), 311),
 (('a', 'little', 'bit', 'of'), 298),
 (('to', 'be', 'able', 'to'), 291),
 (('you', 'know', 'i', 'mean'), 290),
 (('you', 'know', 'i', 'think'), 276),
 (('it', 'was', 'it', 'was'), 275),
 (('you', 'know', 'i', "don't"), 265),
 (('and', 'i', "don't", 'know'), 258),
 (('i', 'mean', 'you', 'know'), 242),
 (('and', 'i', 'think', 'that'), 240),
 (('one', 'of', 'the', 'things'), 239),
 (('and', 'a', 'lot', 'of'), 230),
 (('do', 'a', 'lot', 'of'), 228),
 (('<

In [111]:
contexts_trigram = [tuple(c) for c in local_contexts if len(c) == 3]
len(contexts_trigram)

156659

In [112]:
Counter(contexts_trigram).most_common()

[(('i', "don't", 'know'), 1027),
 (('yeah', 'i', 'think'), 493),
 (('you', 'know', 'i'), 449),
 (('well', 'i', 'think'), 396),
 (('and', 'you', 'know'), 334),
 (('you', 'know', 'and'), 333),
 (('and', 'i', 'think'), 332),
 (('and', 'uh', 'i'), 299),
 (('yeah', 'well', 'i'), 286),
 (('uh', 'you', 'know'), 277),
 (('well', 'you', 'know'), 251),
 (('i', "don't", 'think'), 215),
 (('what', 'do', 'you'), 197),
 (('well', 'i', "don't"), 193),
 (('yeah', 'yeah', 'i'), 191),
 (('you', 'know', 'the'), 190),
 (('yeah', '<rem>', 'i'), 189),
 (('you', 'know', "it's"), 183),
 (('but', 'uh', 'i'), 182),
 (('and', 'uh', 'you'), 182),
 (('yeah', 'i', 'i'), 181),
 (('do', 'you', 'have'), 181),
 (('a', 'lot', 'of'), 178),
 (('yeah', 'i', "don't"), 177),
 (('yeah', 'i', 'know'), 175),
 (('well', 'i', 'guess'), 172),
 (('i', 'think', 'that'), 171),
 (('and', 'it', 'was'), 167),
 (('but', 'i', 'think'), 161),
 (('and', 'i', "don't"), 159),
 (('and', 'uh', 'so'), 159),
 (('yeah', 'i', 'mean'), 158),
 (('uh'

In [113]:
contexts_bigram = [tuple(c) for c in local_contexts if len(c) == 2]
len(contexts_bigram)

166939

In [114]:
Counter(contexts_bigram).most_common()

[(('you', 'know'), 4791),
 (('yeah', 'i'), 2940),
 (('and', 'uh'), 2762),
 (('well', 'i'), 2257),
 (('and', 'i'), 2025),
 (('i', "don't"), 1909),
 (('i', 'think'), 1673),
 (('yeah', 'well'), 1319),
 (('oh', 'yeah'), 1260),
 (('oh', 'i'), 1141),
 (('yeah', 'yeah'), 1139),
 (('yeah', "that's"), 1034),
 (('but', 'uh'), 989),
 (('yeah', 'and'), 975),
 (('but', 'i'), 950),
 (('and', 'then'), 947),
 (('i', 'mean'), 908),
 (('so', 'i'), 902),
 (('uh', 'i'), 855),
 (('well', "that's"), 834),
 (('do', 'you'), 789),
 (('and', 'so'), 785),
 (('and', 'they'), 709),
 (('i', 'know'), 681),
 (('yeah', 'it'), 669),
 (('and', 'you'), 652),
 (('and', 'it'), 648),
 (('i', 'guess'), 640),
 (('no', 'i'), 616),
 (('yeah', "it's"), 608),
 (('um', 'i'), 604),
 (('oh', "that's"), 591),
 (("that's", 'right'), 575),
 (('and', 'the'), 558),
 (('<rem>', 'i'), 554),
 (('it', 'was'), 507),
 (('and', 'and'), 503),
 (('well', 'you'), 491),
 (('and', 'we'), 456),
 (('oh', 'well'), 456),
 (('yeah', '<rem>'), 445),
 (('a

In [115]:
contexts_unigram = [tuple(c) for c in local_contexts if len(c) == 1]
len(contexts_unigram)
len(set(contexts_unigram))
len(set(vocabulary))

184785

4521

27559

In [116]:
Counter(contexts_unigram).most_common()

[(('yeah',), 18375),
 (('and',), 17982),
 (('oh',), 12699),
 (('i',), 12686),
 (('well',), 10727),
 (('uh',), 8556),
 (('you',), 6928),
 (('but',), 6374),
 (('so',), 5657),
 (('um',), 3868),
 (("that's",), 3801),
 (('right',), 3167),
 (('no',), 2654),
 (('<rem>',), 2494),
 (('the',), 2491),
 (("it's",), 2396),
 (('it',), 2362),
 (('uh-huh',), 2199),
 (('that',), 2135),
 (('um-hum',), 1996),
 (('they',), 1837),
 (('okay',), 1828),
 (('yes',), 1708),
 (('we',), 1521),
 (('what',), 1333),
 (("i'm",), 1211),
 (('because',), 1209),
 (('a',), 1091),
 (('do',), 1089),
 (('in',), 1001),
 (('is',), 939),
 (('to',), 836),
 (('if',), 775),
 (('my',), 735),
 (('how',), 731),
 (('all',), 666),
 (('or',), 657),
 (('now',), 641),
 (("i've",), 589),
 (('just',), 552),
 (('for',), 531),
 (('he',), 530),
 (('when',), 530),
 (('not',), 503),
 (('are',), 495),
 (('have',), 483),
 (('which',), 473),
 (("they're",), 469),
 (('like',), 432),
 (('there',), 431),
 (('really',), 399),
 (('where',), 396),
 (('as

# Speech rates by speaker

# Word Durations

# Applying Exclusion Criteria

# Writing things to file

We want on-disk representations of 
 1. the pre-processed utterance relation
 2. the pre-processed word relation
 3. a file with just the pre-processed utterances
 4. the vocabulary in the pre-processed utterances
 5. the word analysis relation
 
#FIXME we want information about just those utterances, contexts, and wordforms relevant under exclusion criteria...

## Utterance relation

In [117]:
os.getcwd()

'/mnt/cube/home/AD/emeinhar/fisher-lm'

In [118]:
os.chdir(switchboard_lm_dir)
switchboard_lm_dir
os.listdir()

'/mnt/cube/home/AD/emeinhar/switchboard-lm'

['.gitignore',
 '.ipynb_checkpoints',
 'swbd2003_words_relation.json',
 'Switchboard annotation exploration.ipynb',
 'swbd2003_bigram_contexts.txt',
 'README.md',
 'swbd2003_trigram_contexts.txt',
 'swbd2003_utterances_relation.json',
 'swbd2003_vocabulary.txt',
 'swbd2003_words_analysis_relation.json',
 'swbd2003_fourgram_contexts.txt',
 'swbd2003_utterances.txt',
 'swbd2003_unigram_contexts.txt',
 'Preprocessing Switchboard (2003 release) corpus transcriptions for ease of processing and use with kenlm.ipynb',
 '.git']

In [119]:
swbd_utterances_relation_fn = 'swbd2003_utterances_relation.json'

swbd_utterances_relation_path = os.path.join(switchboard_lm_dir, swbd_utterances_relation_fn)

#~9s on wittgenstein
with open(swbd_utterances_relation_path, 'w', encoding='utf8') as json_file:
    json.dump(utterances, json_file, indent=1, ensure_ascii=False, sort_keys=True)

In [120]:
swbd_utterances_relation_path

'/mnt/cube/home/AD/emeinhar/switchboard-lm/swbd2003_utterances_relation.json'

In [121]:
%cat -n /mnt/cube/home/AD/emeinhar/switchboard-lm/swbd2003_utterances_relation.json | head -60

     1	[
     2	 {
     3	  "utterance_id": "sw3441A-ms98-a-0001",
     4	  "conversation_id": "3441",
     5	  "speaker": "A",
     6	  "start": 0.0,
     7	  "end": 1.008625,
     8	  "duration": 1.008625,
     9	  "speech_raw": "[noise]",
    10	  "speech": "",
    11	  "n_words": 0,
    12	  "has_words": false,
    13	  "has_word_internal_brackets": false
    14	 },
    15	 {
    16	  "utterance_id": "sw3441A-ms98-a-0002",
    17	  "conversation_id": "3441",
    18	  "speaker": "A",
    19	  "start": 1.008625,
    20	  "end": 5.231875,
    21	  "duration": 4.223249999999999,
    22	  "speech_raw": "[silence]",
    23	  "speech": "",
    24	  "n_words": 0,
    25	  "has_words": false,
    26	  "has_word_internal_brackets": false
    27	 },
    28	 {
    29	  "utterance_id": "sw3441A-ms98-a-0003",
    30	  "conversation_id": "3441",
    31	  "speaker": "A",
    32	  "start": 5.231875,
    33	  "end": 12.82125,
    34	  "duration": 7.5893749999999995,
    35	  "speech_raw": "uh invasi

## Word relation

In [122]:
os.getcwd()

'/mnt/cube/home/AD/emeinhar/switchboard-lm'

In [123]:
os.chdir(switchboard_lm_dir)
switchboard_lm_dir
os.listdir()

'/mnt/cube/home/AD/emeinhar/switchboard-lm'

['.gitignore',
 '.ipynb_checkpoints',
 'swbd2003_words_relation.json',
 'Switchboard annotation exploration.ipynb',
 'swbd2003_bigram_contexts.txt',
 'README.md',
 'swbd2003_trigram_contexts.txt',
 'swbd2003_utterances_relation.json',
 'swbd2003_vocabulary.txt',
 'swbd2003_words_analysis_relation.json',
 'swbd2003_fourgram_contexts.txt',
 'swbd2003_utterances.txt',
 'swbd2003_unigram_contexts.txt',
 'Preprocessing Switchboard (2003 release) corpus transcriptions for ease of processing and use with kenlm.ipynb',
 '.git']

In [124]:
words[0]

{'utterance_id': 'sw3441A-ms98-a-0001',
 'conversation_id': '3441',
 'speaker': 'A',
 'start': 0.0,
 'end': 0.00575,
 'duration': 0.00575,
 'speech_raw': '[silence]',
 'speech': '',
 'has_words': False,
 'has_word_internal_brackets': False}

In [125]:
swbd_words_relation_fn = 'swbd2003_words_relation.json'

swbd_words_relation_path = os.path.join(switchboard_lm_dir, swbd_words_relation_fn)

#~80s on wittgenstein
with open(swbd_words_relation_path, 'w', encoding='utf8') as json_file:
    json.dump(words, json_file, indent=1, ensure_ascii=False, sort_keys=True)

In [126]:
swbd_words_relation_path

'/mnt/cube/home/AD/emeinhar/switchboard-lm/swbd2003_words_relation.json'

In [127]:
%cat -n /mnt/cube/home/AD/emeinhar/switchboard-lm/swbd2003_words_relation.json | head -60

     1	[
     2	 {
     3	  "utterance_id": "sw3441A-ms98-a-0001",
     4	  "conversation_id": "3441",
     5	  "speaker": "A",
     6	  "start": 0.0,
     7	  "end": 0.00575,
     8	  "duration": 0.00575,
     9	  "speech_raw": "[silence]",
    10	  "speech": "",
    11	  "has_words": false,
    12	  "has_word_internal_brackets": false
    13	 },
    14	 {
    15	  "utterance_id": "sw3441A-ms98-a-0001",
    16	  "conversation_id": "3441",
    17	  "speaker": "A",
    18	  "start": 0.00575,
    19	  "end": 0.01525,
    20	  "duration": 0.0095,
    21	  "speech_raw": "[noise]",
    22	  "speech": "",
    23	  "has_words": false,
    24	  "has_word_internal_brackets": false
    25	 },
    26	 {
    27	  "utterance_id": "sw3441A-ms98-a-0001",
    28	  "conversation_id": "3441",
    29	  "speaker": "A",
    30	  "start": 0.01525,
    31	  "end": 1.008625,
    32	  "duration": 0.9933750000000001,
    33	  "speech_raw": "[silence]",
    34	  "speech": "",
    35	  "has_words": false,
    36	

## Corpus for a language model

In [128]:
os.getcwd()

'/mnt/cube/home/AD/emeinhar/switchboard-lm'

In [129]:
os.chdir(switchboard_lm_dir)
switchboard_lm_dir
os.listdir()

'/mnt/cube/home/AD/emeinhar/switchboard-lm'

['.gitignore',
 '.ipynb_checkpoints',
 'swbd2003_words_relation.json',
 'Switchboard annotation exploration.ipynb',
 'swbd2003_bigram_contexts.txt',
 'README.md',
 'swbd2003_trigram_contexts.txt',
 'swbd2003_utterances_relation.json',
 'swbd2003_vocabulary.txt',
 'swbd2003_words_analysis_relation.json',
 'swbd2003_fourgram_contexts.txt',
 'swbd2003_utterances.txt',
 'swbd2003_unigram_contexts.txt',
 'Preprocessing Switchboard (2003 release) corpus transcriptions for ease of processing and use with kenlm.ipynb',
 '.git']

In [130]:
utterances[0]
[u for u in utterances if u['has_words'] and u['speech'] == '']
[u for u in utterances if not u['has_words'] and u['speech'] != '']

{'utterance_id': 'sw3441A-ms98-a-0001',
 'conversation_id': '3441',
 'speaker': 'A',
 'start': 0.0,
 'end': 1.008625,
 'duration': 1.008625,
 'speech_raw': '[noise]',
 'speech': '',
 'n_words': 0,
 'has_words': False,
 'has_word_internal_brackets': False}

[]

[]

In [131]:
just_utts_w_words = [u['speech'] for u in utterances if u['has_words']]
len(just_utts_w_words)
num_words = sum([u['n_words'] for u in utterances if u['has_words']])
num_words

248826

3071988

In [132]:
%%capture
#cell magic above suppresses useless output of file.write

swbd2003_utterances_fn = 'swbd2003_utterances.txt'

swbd2003_utterances_path = os.path.join(switchboard_lm_dir, swbd2003_utterances_fn)


with open(swbd2003_utterances_path, 'w') as file:
    #write all lines at once using writelines and this takes <1s
    utterances_w_linebreaks = list(map(lambda l: l + "\n", just_utts_w_words))
    file.writelines(utterances_w_linebreaks)

    #if someone else is using the machine heavily doing other things, this can take upwards of 2m
#     for utterance in just_utts_w_words:
#         file.write("{0}\n".format(utterance));

In [133]:
swbd2003_utterances_path

'/mnt/cube/home/AD/emeinhar/switchboard-lm/swbd2003_utterances.txt'

In [134]:
%cat -n /mnt/cube/home/AD/emeinhar/switchboard-lm/swbd2003_utterances.txt | head -60

     1	uh invasions of privacy um what most annoys me are all sorts of
     2	phone calls trying to to sell me things
     3	<rem> all sorts of hours calling and
     4	especially with it being that i want investment advice that i want this service or that service or the other thing
     5	um
     6	my sense is that something may just have been done about it by congress and <rem> clear
     7	just what went into the
     8	even the final will i think it does something about the random dialers that just dial phone numbers
     9	i guess in some cases they just consider pickup your
    10	your phone and and and not let you use your phone for a while whether you hang up on them or not
    11	um but i thought i saw something about having some kind of provision that people could specify that their phones were
    12	not for commercial use and and uh and uh people businesses would be prohibited uh from calling for <rem> for purposes of of selling you things i'm sure that's true
    13	ah som

## Vocabulary for a language model

In [135]:
os.getcwd()

'/mnt/cube/home/AD/emeinhar/switchboard-lm'

In [136]:
os.chdir(switchboard_lm_dir)
switchboard_lm_dir
os.listdir()

'/mnt/cube/home/AD/emeinhar/switchboard-lm'

['.gitignore',
 '.ipynb_checkpoints',
 'swbd2003_words_relation.json',
 'Switchboard annotation exploration.ipynb',
 'swbd2003_bigram_contexts.txt',
 'README.md',
 'swbd2003_trigram_contexts.txt',
 'swbd2003_utterances_relation.json',
 'swbd2003_vocabulary.txt',
 'swbd2003_words_analysis_relation.json',
 'swbd2003_fourgram_contexts.txt',
 'swbd2003_utterances.txt',
 'swbd2003_unigram_contexts.txt',
 'Preprocessing Switchboard (2003 release) corpus transcriptions for ease of processing and use with kenlm.ipynb',
 '.git']

In [137]:
word_rels_with_speech = [w_rel for w_rel in words if w_rel['has_words']]
len(word_rels_with_speech)
word_rels_with_speech[0]

3071988

{'utterance_id': 'sw3441A-ms98-a-0003',
 'conversation_id': '3441',
 'speaker': 'A',
 'start': 5.787125,
 'end': 6.1595,
 'duration': 0.3723750000000008,
 'speech_raw': 'uh',
 'speech': 'uh',
 'has_words': True,
 'has_word_internal_brackets': False}

In [138]:
[w for w in word_rels_with_speech if w['speech'] == '']

[]

In [139]:
just_word_tokens = [w_rel['speech'] for w_rel in word_rels_with_speech]
len(just_word_tokens)
just_word_tokens[:10]

3071988

['uh',
 'invasions',
 'of',
 'privacy',
 'um',
 'what',
 'most',
 'annoys',
 'me',
 'are']

In [140]:
word_types = set(just_word_tokens)
len(word_types)

27559

In [141]:
{w for w in word_types if ('ah' in w or 'um' in w or 'mm' in w) and len(w) < 5}

{'ah',
 'aha',
 'alum',
 'bah',
 'bahr',
 'blah',
 'bum',
 'bump',
 'bums',
 'crum',
 'cum',
 'dah',
 'drum',
 'dumb',
 'dump',
 'emmy',
 'gah',
 'gum',
 'hum',
 'hump',
 'jump',
 'lump',
 'mums',
 'nah',
 'numb',
 'oahu',
 'onum',
 'plum',
 'pump',
 'rum',
 'scum',
 'shah',
 'sum',
 'sump',
 'sums',
 'swum',
 'um',
 'utah',
 'yeah'}

In [142]:
sorted_vocabulary = sorted(list(word_types))

In [143]:
%%capture
#cell magic above suppresses useless output of file.write

swbd2003_words_fn = 'swbd2003_vocabulary.txt'

swbd2003_words_path = os.path.join(switchboard_lm_dir, swbd2003_words_fn)


with open(swbd2003_words_path, 'w') as file:
    #write all lines at once using writelines and this takes <1s
    words_w_linebreaks = list(map(lambda l: l + "\n", sorted_vocabulary))
    file.writelines(words_w_linebreaks)

#     for w in list(word_types):
#         file.write("{0}\n".format(w));

In [144]:
swbd2003_words_path

'/mnt/cube/home/AD/emeinhar/switchboard-lm/swbd2003_vocabulary.txt'

In [145]:
%cat -n /mnt/cube/home/AD/emeinhar/switchboard-lm/swbd2003_vocabulary.txt | head -60

     1	007
     2	1
     3	100
     4	101
     5	10a
     6	1200
     7	125k
     8	128
     9	13th
    10	13ths
    11	150
    12	1500
    13	2
    14	20/20
    15	2000
    16	21
    17	260
    18	286
    19	287
    20	2ci
    21	302
    22	365
    23	380
    24	386
    25	4-runner
    26	401k
    27	401k's
    28	48
    29	486
    30	49er
    31	49ers
    32	4h
    33	5
    34	500
    35	60
    36	635
    37	635's
    38	69
    39	6s
    40	7-eleven
    41	7-up
    42	7094
    43	747
    44	8088
    45	90210
    46	9050
    47	911
    48	990
    49	<rem>
    50	a
    51	a&e
    52	a&m
    53	a's
    54	a1
    55	aames
    56	aaron
    57	aarp
    58	aba
    59	aback
    60	abacus
cat: write error: Broken pipe


## Word analysis relation

In [146]:
os.getcwd()

'/mnt/cube/home/AD/emeinhar/switchboard-lm'

In [147]:
os.chdir(switchboard_lm_dir)
switchboard_lm_dir
os.listdir()

'/mnt/cube/home/AD/emeinhar/switchboard-lm'

['.gitignore',
 '.ipynb_checkpoints',
 'swbd2003_words_relation.json',
 'Switchboard annotation exploration.ipynb',
 'swbd2003_bigram_contexts.txt',
 'README.md',
 'swbd2003_trigram_contexts.txt',
 'swbd2003_utterances_relation.json',
 'swbd2003_vocabulary.txt',
 'swbd2003_words_analysis_relation.json',
 'swbd2003_fourgram_contexts.txt',
 'swbd2003_utterances.txt',
 'swbd2003_unigram_contexts.txt',
 'Preprocessing Switchboard (2003 release) corpus transcriptions for ease of processing and use with kenlm.ipynb',
 '.git']

In [148]:
swbd_words_analysis_relation_fn = 'swbd2003_words_analysis_relation.json'

swbd_words_analysis_relation_path = os.path.join(switchboard_lm_dir, swbd_words_analysis_relation_fn)

#~60s on wittgenstein
with open(swbd_words_analysis_relation_path, 'w', encoding='utf8') as json_file:
    json.dump(word_analysis_relation, json_file, indent=1, ensure_ascii=False, sort_keys=True)

In [149]:
swbd_words_analysis_relation_path

'/mnt/cube/home/AD/emeinhar/switchboard-lm/swbd2003_words_analysis_relation.json'

In [150]:
%cat -n /mnt/cube/home/AD/emeinhar/switchboard-lm/swbd2003_words_analysis_relation.json | head -60

     1	[
     2	 {
     3	  "utterance_id": "sw3441A-ms98-a-0003",
     4	  "conversation_id": "3441",
     5	  "speaker": "A",
     6	  "orthographic_wordform": "uh",
     7	  "duration": 0.3723750000000008,
     8	  "preceding_4_wordforms": [],
     9	  "preceding_3_wordforms": [],
    10	  "preceding_wordforms": [],
    11	  "dist_from_left_edge": 0,
    12	  "dist_from_right_edge": 12,
    13	  "hasAdjacentPauseOrDisfluency": false,
    14	  "hasAdjacentFilledPause": false,
    15	  "hasClitic": false
    16	 },
    17	 {
    18	  "utterance_id": "sw3441A-ms98-a-0003",
    19	  "conversation_id": "3441",
    20	  "speaker": "A",
    21	  "orthographic_wordform": "invasions",
    22	  "duration": 0.5037499999999993,
    23	  "preceding_4_wordforms": [
    24	   "uh"
    25	  ],
    26	  "preceding_3_wordforms": [],
    27	  "preceding_wordforms": [
    28	   "uh"
    29	  ],
    30	  "dist_from_left_edge": 1,
    31	  "dist_from_right_edge": 11,
    32	  "hasAdjacentPauseOrDisfluenc

## N-gram contexts

In [ ]:
#FIXME these are *not* how contexts should be calculated...

In [151]:
to_string = lambda context: ' '.join(context)

In [152]:
%%capture
#cell magic above suppresses useless output of file.write

fourgrams_switchboard_alpha = list(map(to_string,
                                       sorted(list(set(contexts_fourgram)))))

fourgrams_switchboard_fn = 'swbd2003_contexts_fourgram.txt'

fourgrams_switchboard_fp = os.path.join(switchboard_lm_dir, fourgrams_switchboard_fn)

with open(fourgrams_switchboard_fp, 'w') as file:
    for w in fourgrams_switchboard_alpha:
        file.write("{0}\n".format(w));
        
len(fourgrams_switchboard_alpha)

In [153]:
%%capture
#cell magic above suppresses useless output of file.write

trigrams_switchboard_alpha = list(map(to_string,
                                      sorted(list(set(contexts_trigram)))))

trigrams_switchboard_fn = 'swbd2003_contexts_trigram.txt'

trigrams_switchboard_fp = os.path.join(switchboard_lm_dir, trigrams_switchboard_fn)

with open(trigrams_switchboard_fp, 'w') as file:
    for w in trigrams_switchboard_alpha:
        file.write("{0}\n".format(w));
        
len(trigrams_switchboard_alpha)

In [154]:
%%capture
#cell magic above suppresses useless output of file.write

bigrams_switchboard_alpha = list(map(to_string,
                                     sorted(list(set(contexts_bigram)))))

bigrams_switchboard_fn = 'swbd2003_contexts_bigram.txt'

bigrams_switchboard_fp = os.path.join(switchboard_lm_dir, bigrams_switchboard_fn)

with open(bigrams_switchboard_fp, 'w') as file:
    for w in bigrams_switchboard_alpha:
        file.write("{0}\n".format(w));
        
len(bigrams_switchboard_alpha)

In [155]:
%%capture
#cell magic above suppresses useless output of file.write

unigrams_switchboard_alpha = list(map(to_string,
                                      sorted(list(set(contexts_unigram)))))

unigrams_switchboard_fn = 'swbd2003_contexts_unigram.txt'

unigrams_switchboard_fp = os.path.join(switchboard_lm_dir, unigrams_switchboard_fn)

with open(unigrams_switchboard_fp, 'w') as file:
    for w in unigrams_switchboard_alpha:
        file.write("{0}\n".format(w));
        
len(unigrams_switchboard_alpha)

In [156]:
total_num_contexts = len(unigrams_switchboard_alpha) + len(bigrams_switchboard_alpha) + len(trigrams_switchboard_alpha) + len(fourgrams_switchboard_alpha)
total_num_contexts

1958397

In [157]:
len(sorted_vocabulary)

27559

In [158]:
[unigrams_switchboard_fn, bigrams_switchboard_fn, trigrams_switchboard_fn, fourgrams_switchboard_fn]

['swbd2003_contexts_unigram.txt',
 'swbd2003_contexts_bigram.txt',
 'swbd2003_contexts_trigram.txt',
 'swbd2003_contexts_fourgram.txt']

In [159]:
# !cat swbd2003_contexts_unigram.txt swbd2003_contexts_bigram.txt swbd2003_contexts_trigram.txt swbd2003_contexts_fourgram.txt > swbd2003_contexts.txt

In [164]:
!cat swbd2003_contexts_unigram.txt swbd2003_contexts_bigram.txt swbd2003_contexts_trigram.txt > swbd2003_contexts.txt

In [165]:
!cat -n swbd2003_contexts.txt | head -10

     1	60
     2	<rem>
     3	a
     4	abby
     5	abc
     6	ability
     7	able
     8	about
     9	above
    10	abraham
cat: write error: Broken pipe


In [166]:
!wc -l swbd2003_contexts.txt

108397 swbd2003_contexts.txt


In [167]:
len(unigrams_switchboard_alpha) + len(bigrams_switchboard_alpha) + len(trigrams_switchboard_alpha) + len(fourgrams_switchboard_alpha)
len(unigrams_switchboard_alpha) + len(bigrams_switchboard_alpha) + len(trigrams_switchboard_alpha)# + len(fourgrams_switchboard_alpha)

1958397

108397